patterns modulo theories
ZF using Bohua as guide

In [ ]:
EInt = kd.Inductive("EInt")
EInt.declare("Nil")
EInt.declare("Pow", ("pow", smt.ArraySort(smt.IntSort(), smt.DatatypeSort("EInt"))))
EInt.create()

smt.Array( , smt.IntSort())

EInt

# Untyped knuckledragger

Using lark.Tree for serious as a polymoprhic term to store.
We could instantiate to Opaque type vraibles and check properties

class PolyProof
    thm : lark.Tree

class PolyProof
    thm : str

class PolyExpr:
    def __add__(self), other:
         PolyExpr("{self.expr} + {other.expr}")
    def __sub__(self):
        ...

What's the point?
def Foo(T):
    return ...
A furst class insepctalbe versoin


def instan(tree, T):
    match tree
        case Tree("forall", ):
            s.add()
            


# smtmsmt smtmt automata
Automata minimizatiom modulo theories


E-antiunification?
Not(Distinct)
Not(And(equal))
And(equal)
Distinct

If there is any model where they are the same, merge them.
If they are the same in all models, merge them
if there is any model where they are different, partition them


In [13]:
from kdrag.all import *
from collections import defaultdict
State = smt.DeclareSort("State")
Obs = kd.Struct("Obs", ("head", smt.IntSort()), ("tl", State))
obs = smt.Function("obs", State, Obs)

def minimize(states, constrs):
    s = smt.Solver()
    s.add(constrs)
    sets = [states]
    #s.add(smt.Not(smt.Distinct(states)))
    while True:
        #s.add(smt.Or(smt.Not(smt.Distinct(group))) for group in sets)
        s.add(smt.And())
        res = s.check()
        if res == smt.unsat:
            return sets
        assert res == smt.sat
        m = s.model()
        newsets = []
        for group in sets:
            d = defaultdict(list)
            for st in group:
                d[m.eval(obs(st)).get_id()].append(st)
            newsets.extend(d.values())



a,b,c = smt.Consts("a b c", State)


states = [a,b,c]
constrs = [
obs(a) == Obs(1, b),
obs(b) == Obs(2, a),
obs(c) == Obs(1, b),
]

#minimize([a,b], constrs)

def prop_diseq(states, constrs):
    diseqs = [st1 != st2 for i, st1 in enumerate(states) for st2 in states[i+1:]]
    return kd.utils.propagate(diseqs, constrs)



print(kd.utils.propagate_eqs(states, constrs))
print(prop_diseq(states, constrs))


[]
[a != b, b != c]


In [ ]:

            sorts = sorted([(m.eval(st).get_id(), st) for st in group], key=lambda x: x[0])
            for subgroup in itertools.groupby(sorts, key=lambda x: x[0]):
                newgroup.extend([a for _, a in subgroup[1]])
                newgroups.append(newgroup)
                newsets[tuple(newgroup)] = obs(newgroup[0])
        [group in groups for subgroup in itertools.groupby(sorted((m.eval(a).get_id(), a)for a in group),]  
        for o, part in sets.items():
            m.eval(obs(st)) #m.eval(s) ?

# export



In [19]:
from kdrag.all import *
from dataclasses import dataclass, field
def module(decls=[], thms=[]):
    module = smt.Function("module", smt.BoolSort(), smt.BoolSort())
    defn = smt.Function("define",smt.BoolSort(), smt.BoolSort())
    theorem = smt.Function("theorem", smt.BoolSort(), smt.BoolSort())
    opaque = smt.Function("opaque", smt.BoolSort(), smt.BoolSort())
    body = []
    for decl in decls:
        body.append(defn(decl.defn.thm))
    for thm in thms:
        assert isinstance(thm, kd.Proof)
        body.append(theorem(thm.thm))
    
    return module(smt.And(body))

def decl_wrap(f : smt.FuncDeclRef) -> smt.BoolSort:
    decl = smt.Function("decl", smt.BoolSort(), smt.BoolSort())
    return smt.ForAll(vs, decl(*vs) == decl(*vs)) # smt.Distinct?

@dataclass
class Module():
    decls : list[smt.FuncDeclRef] = field(default_factory=list)
    theorems : list[kd.Proof] = field(default_factory=list)


    module = smt.Function("module", smt.BoolSort(), smt.BoolSort())
    defn = smt.Function("define",smt.BoolSort(), smt.BoolSort())
    theorem = smt.Function("theorem", smt.BoolSort(), smt.BoolSort(), smt.BoolSort())
    opaque = smt.Function("opaque", smt.BoolSort(), smt.BoolSort())
    prove = smt.Function("prove", smt.BoolSort(), smt.BoolSort(), smt.BoolSort())
    axiom = smt.Function("axiom", smt.BoolSort(), smt.BoolSort())
    unfold = smt.Function("unfold", smt.BoolSort(), smt.BoolSort())

    def serialize(self):

        body = []
        for decl in self.decls:
            body.append(Module.defn(decl.defn.thm))
        for thm in self.theorems:
            assert isinstance(thm, kd.Proof)
            body.append(Module.axiom(thm.thm)) 
        s = smt.Solver()
        s.add(Module.module(smt.And(body)))
        return s.sexpr()
    @classmethod
    def deserialize(cls, st : str):
        s = smt.Solver()
        s.from_string(st)
        m = cls()
        if len(s.assertions()) != 1:
            raise Exception("single assertion expected")
        module = s.assertions()[0]
        if module.decl() != Module.module:
            raise Exception("function 'module' expected", module.decl())
        body = module.arg(0)
        assert smt.is_and(body)
        for stmt in body.children():
            decl = stmt.decl()
            args = stmt.children()
            if decl == Module.defn:
                assert len(args) == 1
                vs, body = kd.utils.open_binder_unhygienic(args[0])
                assert smt.is_eq(body)
                f,body = body.children()
                m.decls.append(kd.define(f.decl().name(), vs, body))
            elif decl == Module.axiom:
                m.theorems.append(kd.axiom(args[0]))
            else:
                raise Exception("Unexpected decl", decl)
        return m




x,y = smt.Ints("x y")
myadd = kd.define("myadd", [x,y], x + y)
mythm = kd.prove(myadd(1,2) == 3, unfold=1)
module([myadd, myadd], [mythm]).serialize()



m = Module()
m.decls.append(myadd)
m.theorems.append(mythm)
Module.deserialize(m.serialize())


Module(decls=[myadd], theorems=[|= myadd(1, 2) == 3])

# cegraph
I need some data structures

Could use And trees.

`propagate` of equality between roots would be faster.
Also intertwined with a union find could make the next iteration smaller.
maybe interlave checks with building all binary pairs. Doing all binary pairs then propagating is inefficient.



# ATL
A tensor tanguage
Can we do these rewrites in smt?
# Schroder Bernstein


In [ ]:
def Tarski(join, meet):
    l = CompleteLattice(bigjoin)
    l = Lemma(smt.ForAll([x,y], ...))
    return l.qed()


# minikanren parse


In [ ]:
import kdrag.parsers.sexp as sexp

sexp.parse("""
(defrel appendo (x y z) 
    (conde
        ((== nil x) (== y z))
        ((fresh (h t r)
            (== (cons h t) x)
            (== (cons h r) z)
            (appendo t y r)))))

(run 5 (q)
    (appendo (cons 1 (cons 2 nil)) (cons 3 (cons 4 nil)) q))
""")

def expr(p):
    if isinstance(p, str):
        return smt.Const(p, kd.U)
    elif isinstance(p, int):
        return smt.IntVal(p)
    elif isinstance(p, list):
        match p:
            case ["==", a, b]:
                return expr(a) == expr(b)
            case ["conde" , *clauses]:
                return smt.Or([smt.And([expr(c) for c in clause]) for clause in clauses])
            case ["fresh", vars, *body]:
                smt_vars = [smt.Const(v, kd.U) for v in vars]
                return smt.Exists(smt_vars, smt.And([expr(b) for b in body]))
            case [f, *args]:
                f = smt.Function(f, *[kd.U for _ in args], kd.U)
                return f(*[expr(a) for a in args])
        


def stmts(ps):
    defns = []

    for p in ps:
        match p:
            case ["defrel", name, args, *body]:
                f = smt.Function(name, *[kd.U for _ in args], kd.U)
                args = [smt.Const(a, kd.U) for a in args]
                defns.append(kd.define(name, args, expr(body)))
            case ["run", n, vs, qs]:
                vs = [smt.Const(v, kd.U) for v in vs]
                kd.esimp(vs, smt.And([expr(q) for q in qs]))


[['defrel',
  'appendo',
  ['x', 'y', 'z'],
  ['conde',
   [['==', 'nil', 'x'], ['==', 'y', 'z']],
   [['fresh',
     ['h', 't', 'r'],
     ['==', ['cons', 'h', 't'], 'x'],
     ['==', ['cons', 'h', 'r'], 'z'],
     ['appendo', 't', 'y', 'r']]]]]]

# lambda lift


# release notes

Make an api key.

uv version --bump minor
uv build
uv publish
__token__

Pretty stragithforward really
I'm using uv build now?
Needed to exclude lots of files.
Kind of makes it seems like it would be easy to include nice juicy binaries.




# prelude
wire up the cheap to import stuff to be avaiable by default.
Maybe not piles of theorems
But helper constrructors, useful datatypes

Wire up seq to have induction principle on by default
special case set operations in overloading

Bring NatSort() up



# isabelle why3

export isabelle or whyml to smtlib?
https://github.com/inpefess/isabelle-client

https://github.com/ejgallego/pycoq

acl2-bridge

pantograph
https://github.com/LLM4Rocq/pytanque

In [25]:
%%file /tmp/test.mlw
theory Test
use import int.Int
let f (x: int) : int
  ensures { result >= x }
= x + 1
end


Writing /tmp/test.mlw


In [26]:
!why3 prove --smtdir tmp test.mlw

why3 prove: unrecognized option '--smtdir'
Try 'why3 prove --help' for more information.


# cvc5
interpolants
get-qe
synth

In [ ]:
def to_cvc5_sort(tm, s : smt.SortRef):
    if s == smt.BoolSort():
        return tm.getBoolSort()
    elif s == smt.IntSort():
        return tm.getIntegerSort()
    else:
        raise NotImplementedError(f"Unsupported sort: {s}")


def of_decl(tm, decl : smt.FuncDeclRef):
    name = decl.name()
    domain_sorts = [to_cvc5_sort(tm, decl.domain(i)) for i in range(decl.arity())]
    range_sort = to_cvc5_sort(tm, decl.range())
    func_sort = tm.mkFunctionSort(domain_sorts, range_sort)
    return tm.mkConst(func_sort, name)

def to_cvc5(tm, e : smt.ExprRef):
    if smt.is_app(e):
        decl = e.decl()
        args = [to_cvc5(tm, arg) for arg in e.children()]
        tm.mkTerm(cvc5.Kind.APPLY_UF, )

    #  quantifiedVariables = tm.mkTerm(Kind.VARIABLE_LIST, x)
    # noSelfAncestor = tm.mkTerm(Kind.FORALL, quantifiedVariables, notMember)

        


In [9]:
s = smt.Solver()
s.add(x == x + x)

slv = cvc5.Solver(tm)
parser = cvc5.InputParser(slv)
sm = parser.getSymbolManager()
inp = "(set-logic ALL)" + s.sexpr() + "(check-sat)"
print(s.sexpr())
parser.setStringInput(cvc5.InputLanguage.SMT_LIB_2_6, inp, "MyInput")
while True:
        cmd = parser.nextCommand()
        if cmd.isNull():
            break

(declare-fun x () Int)
(assert (= x (+ x x)))



RuntimeError: Symbol 'x' not declared as a variable

In [4]:
import cvc5
from kdrag.all import *

x,y,z = smt.Ints("x y z")

tm = cvc5.TermManager()
slv = cvc5.Solver(tm)

# set that we should print success after each successful command
slv.setOption("print-success", "true")

# construct an input parser associated the solver above
parser = cvc5.InputParser(slv)
parser.setStringInput(cvc5.InputLanguage.SMT_LIB_2_6, (x+x).serialize(), "MyInput")

cmd = parser.nextCommand()
cmd.invoke()
#dir(cmd)
#cmd.getCommandName()
# #parser.nextTerm()

MyInput:1.2: No set-logic command was given before this point.
MyInput:1.2: cvc5 will make all theories available.
MyInput:1.2: Consider setting a stricter logic for (likely) better performance.
MyInput:1.2: To suppress this warning in the future use (set-logic ALL).


TypeError: invoke() takes exactly 2 positional arguments (0 given)

# vampire
answer


In [56]:
%%file /tmp/test.smt2
(set-logic UF)
(assert-synth () ((x Bool)) (= x true))

Overwriting /tmp/test.smt2


In [57]:
! vampire --input_syntax smtlib2 -qa synthesis /tmp/test.smt2

% Refutation found. Thanks to Tanya!
% SZS status Unsatisfiable for test
% SZS output start Proof for test
1. ! [] : ? [X0 : $o] : (X0 = $true) [input(conjecture)]
2. ~! [] : ? [X0 : $o] : (X0 = $true) [negated conjecture 1]
3. ~? [X0 : $o] : ((X0 = $true) & ans0(X0)) [answer literal with input var skolemisation 2]
6. ~? [X0 : $o] : ((($true = X0) <=> $true) & ans0(X0)) [fool elimination 3]
7. ~? [X0 : $o] : (($true = X0) & ans0(X0)) [true and false elimination 6]
8. ! [X0 : $o] : (($true != X0) | ~ans0(X0)) [ennf transformation 7]
9. ($true != X0) | ~ans0(X0) [cnf transformation 8]
10. ~ans0($true) [equality resolution 9]
11. ans0(X0) [answer literal resolver]
12. $false [unit resulting resolution 10,11]
% SZS output end Proof for test
% SZS answers Tuple [[$true]|_] for test
% ------------------------------
% Version: Vampire 5.0.0 (Release build, commit 55c27f5 on 2025-09-09 16:15:20 +0100)
% Linked with Z3 4.14.0.0 3c47fd96cf5645d0c42b2c819d9e9a84380aa721 NOTFOUND
% Termination rea

In [38]:
%%file /tmp/test.smt2
(set-logic LIA)

(assert-synth (
    (x1 Int)
    (x2 Int)
    (x3 Int)
    (x4 Int)
    (x5 Int))
    ((y Int))
        (and
            (>= y x1)
            (>= y x2)
            (>= y x3)
            (>= y x4)
            (>= y x5)
            (or
                (= y x1)
                (= y x2)
                (= y x3)
                (= y x4)
                (= y x5)
            )
))


Overwriting /tmp/test.smt2


In [39]:
! vampire --input_syntax smtlib2 -qa synthesis /tmp/test.smt2

% Refutation found. Thanks to Tanya!
% SZS status Unsatisfiable for test
% SZS output start Proof for test
1. ! [X4 : $int,X3 : $int,X2 : $int,X1 : $int,X0 : $int] : ? [X5 : $int] : ($greatereq(X5,X0) & $greatereq(X5,X1) & $greatereq(X5,X2) & $greatereq(X5,X3) & $greatereq(X5,X4) & (X0 = X5 | X1 = X5 | X2 = X5 | X3 = X5 | X4 = X5)) [input(conjecture)]
2. ~! [X4 : $int,X3 : $int,X2 : $int,X1 : $int,X0 : $int] : ? [X5 : $int] : ($greatereq(X5,X0) & $greatereq(X5,X1) & $greatereq(X5,X2) & $greatereq(X5,X3) & $greatereq(X5,X4) & (X0 = X5 | X1 = X5 | X2 = X5 | X3 = X5 | X4 = X5)) [negated conjecture 1]
3. ~? [X5 : $int] : (($greatereq(X5,sK5_in) & $greatereq(X5,sK4_in) & $greatereq(X5,sK3_in) & $greatereq(X5,sK2_in) & $greatereq(X5,sK1_in) & (sK5_in = X5 | sK4_in = X5 | sK3_in = X5 | sK2_in = X5 | sK1_in = X5)) & ans0(X5)) [answer literal with input var skolemisation 2]
4. ~? [X5 : $int] : (~$less(X5,sK5_in) & ~$less(X5,sK4_in) & ~$less(X5,sK3_in) & ~$less(X5,sK2_in) & ~$less(X5,sK1_in) & (

In [8]:
%%file /tmp/test.smt2
(set-logic UF)

(declare-sort s 0)

(declare-fun inv (s) s)
(declare-fun op (s s) s)

(declare-const e s)

; Left inverse
(assert (forall ((x s)) (= (op (inv x) x) e)))
; Left identity
(assert (forall ((x s)) (= (op e x) x)))
; Associativity
(assert (forall ((x s) (y s) (z s)) (= (op x (op y z)) (op (op x y) z))))

; Conjecture: inverse of products equals the product of the inverse, in opposite order
(assert-synth ((x s) (y s)) ((z s)) (= (op z (op (inv x) (inv y))) e))

(set-option :uncomputable (inv))

Overwriting /tmp/test.smt2


In [35]:
from kdrag.all import *

s = smt.Solver()
x,y = smt.Ints("x y")
s.add(smt.Lambda([x], x) == smt.Lambda([y], y))
s.sexpr()

'(assert (= (lambda ((x Int)) x) (lambda ((y Int)) y)))\n'

In [46]:
print(smt.Array("x", smt.IntSort(), smt.IntSort(), smt.IntSort()).serialize())

(declare-fun F ((Array Int Int Int)) Bool)
(declare-fun x () (Array Int Int Int))
(assert (F x))



Nothing accepts uncurried arrays.


In [ ]:
def curry_array(e):
    """
    Possibility of clashing names here.
    """
    if isinstance(e, smt.QuantifierRef):
        vs, body = kd.utils.open_binder_unhygienic(e)
        body = curry_array(body)
        if e.is_lambda():
            for v in reversed(vs):
                body = smt.Lambda([v], body)
            return body
        elif e.is_forall():
            return smt.ForAll(vs, body)
        elif e.is_exists():
            return smt.Exists(vs, body)
    elif smt.is_const(e):
        if isinstance(e, smt.ArrayRef):
            doms = smt.domains(e)
            if len(doms) == 1:
                return e
            else:
                sort = e.range()
                for d in reversed(doms):
                    sort = smt.ArraySort(d, sort)
                return smt.Const(e.decl().name(), sort)
        else:
            return e
    elif smt.is_app(e):
        f, children = e.decl(), e.children()
        children = [curry_array(c) for c in children]
        if smt.is_select(e):
            arr = children[0]
            for index in children[1:]:
                arr = smt.Select(arr, index)
            return arr
        else:
            return f(*children)


f = smt.Const("f", smt.ArraySort(smt.IntSort(), smt.IntSort(), smt.IntSort()))
curry_array(f[1,2])




f[1][2]

In [58]:
%%file /tmp/vamp.smt2
(set-logic ALL)
(declare-fun F ((Array Int (Array Int Int))) Bool)
(declare-fun x () (Array Int (Array Int Int)))
(assert (F x))
;(assert (= (lambda ((x Int)) x) (lambda ((y Int)) y)))
(check-sat)


Overwriting /tmp/vamp.smt2


In [61]:
! vampire /tmp/vamp.smt2

% Running in auto input_syntax mode. Trying SMTLIB2
% SZS status Satisfiable for vamp
% SZS output start Saturation.
% SZS output end Saturation.
% SZS output start Definitions and Model Updates.
for all inputs,
    define 'F'(X0) := $true
% SZS output end Definitions and Model Updates.
% ------------------------------
% Version: Vampire 5.0.0 (Release build, commit 55c27f5 on 2025-09-09 16:15:20 +0100)
% Linked with Z3 4.14.0.0 3c47fd96cf5645d0c42b2c819d9e9a84380aa721 NOTFOUND
% Termination reason: Satisfiable

% Time elapsed: 0.019 s
% Peak memory usage: 143 MB
% ------------------------------
% ------------------------------


In [ ]:
import hashlib
import os
from pathlib import Path
import requests
from tqdm import tqdm

def sha256_file(path: Path, chunk_size: int = 1024 * 1024) -> str:
    """Compute SHA256 of a file in streaming fashion."""
    h = hashlib.sha256()
    with open(path, "rb") as f:
        for chunk in iter(lambda: f.read(chunk_size), b""):
            h.update(chunk)
    return h.hexdigest()

def download_with_checksum(
    url: str,
    dest: str | Path,
    expected_sha256: str,
    chunk_size: int = 256 * 1024,
) -> Path:
    directory = Path(__file__).parent()
    dest = Path(dest)
    dest.parent.mkdir(parents=True, exist_ok=True)

    # 1. If file exists, verify its checksum
    if dest.exists():
        actual = sha256_file(dest)
        if actual.lower() == expected_sha256.lower():
            return dest
        else:
            dest.unlink()

    tmp_dest = dest.with_suffix(dest.suffix + ".part")

    # 2. Download with progress bar
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        total = int(r.headers.get("content-length", 0))

        with open(tmp_dest, "wb") as f, tqdm(
            total=total if total > 0 else None,
            unit="B",
            unit_scale=True,
            unit_divisor=1024,
            desc=f"Downloading {dest.name}",
        ) as bar:
            for chunk in r.iter_content(chunk_size=chunk_size):
                if not chunk:
                    continue
                f.write(chunk)
                bar.update(len(chunk))

    # 3. Verify downloaded file’s checksum
    actual = sha256_file(tmp_dest)
    if actual.lower() != expected_sha256.lower():
        tmp_dest.unlink(missing_ok=True)
        raise ValueError(
            f"Checksum mismatch for downloaded file!\n"
            f"expected: {expected_sha256}\nactual:   {actual}"
        )

    # 4. Promote .part → final
    os.replace(tmp_dest, dest)
    return dest


In [12]:
! vampire -qa synthesis /tmp/test.smt2

% Running in auto input_syntax mode. Trying SMTLIB2
% Failed with
% User error: unrecognized logic ALL_SUPPORTED ( use `--ignore_unrecognized_logic on` if you want vampire to try proof search anyways) in top-level expression '(set-logic ALL_SUPPORTED)'
% Trying TPTP
parse error in "", line 1: cnf(), fof(), vampire() or include() expected (text: ()


# numpy
good old times. destructuring


1d- discrte time kalman filter


In [ ]:
from kdrag.all import *
import numpy as np
import math


def from_np(a : np.ndarray):
    assert all(math.log2(x).is_integer() for x in a.shape), "only power-of-two shapes supported"
    dom = [smt.BitVecSort(int(math.log2(i))) for i in a.shape]
    
    #res = smt.K(dom, a[0]) #? no multiarity?
    res = smt.Lambda([smt.Const("i"+str(d), dom[d]) for d in range(len(dom))], smt.RealVal(0))
    for idx in np.ndindex(a.shape):
        res = smt.Store(res, *(smt.BitVecVal(i, int(math.log2(a.shape[d]))) for d, i in enumerate(idx)), a[idx])
    return res

def to_np(x : smt.FuncRef):
    doms = smt.domains(x)
    shape = tuple(2**d.size() for d in doms)
    res = np.zeros(shape, dtype=object)
    for idx in np.ndindex(shape):
        res[tuple(idx)] = smt.simplify(x[idx])
    return res

from_np(to_np(from_np(np.zeros((4,4)))))
x = to_np(smt.Array("x", smt.BitVecSort(2), smt.BitVecSort(2), smt.RealSort()))

x,y,z = map(to_np,smt.Consts("x y z", smt.ArraySort(smt.BitVecSort(2), smt.BitVecSort(2), smt.RealSort())))

def NDConst(name, *shape):
    assert all(math.log2(x).is_integer() for x in shape), "only power-of-two shapes supported"
    return to_np(smt.Array(name, *(smt.BitVecSort(int(math.log2(s))) for s in shape), smt.RealSort()))
def NDConsts(names, *shape):
    return [NDConst(name, *shape) for name in names.split()]


x,y,z = NDConsts("x y z", 4, 4)

kd.prove(from_np(x @ (y @ z)) == from_np((x @ y) @ z))



|= Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Lambda([i0,
                                        i1],
                                        0),
                                        0,
                                        0,
                                        Select(x, 0, 0)*
                                        (...*... +
                                        ...*... +
                                        ...*... +
                                        ...*...) +
                                        Select(x, 0, 1)*
                                        (...*... +
                                        ...*... +
                                        ...*... +
                                        ...*...) +
                                        Select(x, 0, 2)*
                                        (...*... +
                                        ...*... +
                                        ...*... +
                                        ...*...) +
                                        Select(x, 0, 3)*
                                        (...*... +
                                        ...*... +
                                        ...*... +
                                        ...*...)),
                                        0,
                                        1,
                                        Select(x, 0, 0)*
                                        (Select(...,
                                        ...,
                                        ...)*
                                        Select(...,
                                        ...,
                                        ...) +
                                        Select(...,
                                        ...,
                                        ...)*
                                        Select(...,
                                        ...,
                                        ...) +
                                        Select(...,
                                        ...,
                                        ...)*
                                        Select(...,
                                        ...,
                                        ...) +
                                        Select(...,
                                        ...,
                                        ...)*
                                        Select(...,
                                        ...,
                                        ...)) +
                                        Select(x, 0, 1)*
                                        (Select(...,
                                        ...,
                                        ...)*
                                        Select(...,
                                        ...,
                                        ...) +
                                        Select(...,
                                        ...,
                                        ...)*
                                        Select(...,
                                        ...,
                                        ...) +
                                        Select(...,
                                        ...,
                                        ...)*
                                        Select(...,
                                        ...,
                                        ...) +
                                        Select(...,
                                        ...,
                                        ...)*
                                        Select(...,
                                        ...,
                                        ...)) +
                                        Select(x, 0, 2)*
                                        (Select(...,
                                        ...,
                       

In [ ]:
@functools.cache
def FinSort(N):
    """
    Make a finite enumeration.

    >>> FinSort(3)
    Fin3
    """
    return kd.Enum("Fin" + str(N), " ".join(["X" + str(i) for i in range(N)]))



In [55]:
np.diff(np.zeros(4))

array([0., 0., 0.])

# reflect
"Wouldn't proof by reflection be an example of a technique that relies on the computational content of the proof objects, and therefore can't be done as neatly in an LCF-style system?

(Proof by reflection is essentially a decision procedure that has been proved to be sound within the system, it was used crucially in Georges Gonthier's proof of the four-colour theorem.)"

I've been sort of in the game of thinking about what needs dependent types or not. I think there are interesting analogs that do not require dependent types, but neatness is arguable. `exec(t) = true -> P` is the theorem we use.
Dependent type systems need to use metaprogramming to derive t from P afaik. They can derive P from t internal to the system.

1 + 2 = 3 + 4  too easy

forall x,y,z, a + b + c = c + b + a
(ring tactic)
exec(Eq(t1,t2)) = true -> P

one variable
forall x, 2 + x + 1 + 3 = 4 + 1 + x

interp(Add(2,Add(V, 3))) = lam x, const(2)(x) + interp(Add)(x)

prop(t) = forall x, interp(t)(x)

This forall is non computational, but so what

exec(t) = true -> prop(t)


but is this what we're taLKING ABOUT or are we talking about nbe. Are they the same?

I've been sort of in the game of thinking about what needs dependent types or not. I think reflection does not require dependent types, but neatness is arguable.

The basic thing you need is a theorem of the form exec(t) = true -> prop(t). Even in type theory, deriving a t from a proposition P such that prop(t) = P requires some metaprogramming even if it's obvious. prop(t) is a type that depends on a value in the type theory formalism, but might only be a boolean valued function (pretty mundane) in another.

In type theory with its notion of definitional equality (which could be uncharitably described as an automated theorem prover for a decidable equational theory baked in the kernel), deriving exec(t) = true may then be considered free/automatic. It will also possibly be fast because such a deep part of the system receives lots of love from the developers. But in another system without such an equality baked in, making a reliable conversion tactic to execute a deterministic system may not be that hard. If in fact exec were an interpreter of x86 say and t an assembly program, the possibility of using extraction to low level code presents itself if you're willing to take that process into your trusted codebase. To some degree Rocq and Lean do something similar with native_compute and lean's compilation through C as I understand it.

I usually find I have a cycle of understanding why two things are the same and then why they are distinct and then back to why they are the same. I'm on the part of the cycle of understanding why really dependent type theory isn't as critical a part of various logical concepts as I once thought.







Extensible typing typeguard


# minikanren eunify

qelim-light or heavy
z3 can unfold under stuff. Proof objects can be returned.
simp with vars



In [33]:
from kdrag.all import *
from kdrag.rewrite import *
def esimp(vs, goal, ctx=[], max_iters=7, timeout=100):
    # timeout?
    for i in range(max_iters):
        goal = smt.simplify(goal)
        goal = unfold(goal) # unfold in context too?
        #goal = smt.And(*smt.Tactic("nnf")(goal)[0])
        #print(goal)
        #goal = smt.Tactic("elim-predicates")(goal)[0][0]
        print(goal)
        s = smt.Solver()
        s.set("timeout", timeout)
        s.add(ctx)
        s.add(goal)
        res = s.check()
        if res == smt.unsat:
            return False # Can't be satisfied. raise?
        elif res == smt.sat:
            # Do loop or generalize here?
            m = s.model()
            subst = {v: m.eval(v) for v in vs}
            print(subst)
            sgoal = smt.substitute(goal, *subst.items())
            s = smt.Solver()
            s.set("timeout", timeout)
            s.add(ctx)
            s.add(smt.Not(sgoal))
            res = s.check()
            if res == smt.unsat:
                return subst
            elif res == smt.sat:
                pass
            else:
                pass
                # still satisfiable. Not a provable solution.   
        else:
            pass # unknown raise?

    #goal = smt.Tactic("euf-completion")(goal)[0][0]



x,y = smt.Ints("x y")
fact = smt.Function("fact", smt.IntSort(), smt.IntSort())
fact = kd.define("fact", [x], smt.If(x <= 0, 1, x * fact(x - 1)))
esimp([x], fact(x) == 6)

myid = kd.define("myid", [x], x)
assert not esimp([x], myid(x) == x + 1)

f 
f = smt.Array("f", smt.IntSort(), smt.IntSort())
g = smt.Array("g", smt.IntSort(), smt.IntSort())

esimp([f],smt.ForAll([y], f(y) == y + 1))



If(x <= 0, 1, x*fact(x - 1)) == 6
{x: 6}
And(Not(x <= 0),
    x*If(-1 + x <= 0, 1, (-1 + x)*fact(-1 + x - 1)) == 6)
{x: 2}
And(Not(x <= 0),
    x*
    If(x <= 1,
       1,
       If(-2 + x <= 0, 1, (-2 + x)*fact(-2 + x - 1))*
       (-1 + x)) ==
    6)
{x: 3}
And(Not(x <= 0),
    x*
    If(x <= 1,
       1,
       (-1 + x)*
       If(x <= 2,
          1,
          If(-3 + x <= 0, 1, (-3 + x)*fact(-3 + x - 1))*
          (-2 + x))) ==
    6)
{x: 3}
x == 1 + x
ForAll(y, f[y] == 1 + y)
ForAll(y, f[y] == 1 + y)
ForAll(y, f[y] == 1 + y)
ForAll(y, f[y] == 1 + y)
ForAll(y, f[y] == 1 + y)
ForAll(y, f[y] == 1 + y)
ForAll(y, f[y] == 1 + y)


In [ ]:
from kdrag.all import *
x = smt.Int("x")

Even = kd.InductiveRel("Even", x)
Even.declare("SS", ("subpf", Even), pred=lambda subpf: smt.And(x >= 0, subpf(x - 2)))
Even.declare("Z", pred=lambda: x == 0)
Even = Even.create()

pf = smt.Const("mypf", Even)
kd.rewrite.esimp([pf], Even(6)[pf])


{mypf: SS(SS(SS(Z)))}

Uninterpreted sorts - Take model. guess equalities. Refine
This is a lot like antiunification


eqaultiy propagation

Maybe could get this info out of horn solver? Interpolation

Propagation will only keep the stuff it can prove. We may want to branch split to complete a substitution.





In [ ]:
import kdrag.solvers.sympy as sympy
from collections import defaultdict
def propagate(maybes : list[smt.BoolRef], goal : smt.BoolRef):
    s = smt.Solver()
    s.add(goal)
    maybes = list(maybes)
    while (res := s.check()) != smt.unsat:
        assert res == smt.sat
        m = s.model()
        maybes = [e for e in maybes if m.eval(e)]
        s.add(smt.Not(smt.And(maybes)))
    return maybes
def propagate_eqs(terms, goal): # propagate eqs
    # sort by sorts
    ts = defaultdict(list)
    for t in terms:
        ts[t.sort()].append(t)
    eqs = [t1 == t2 for terms in ts.values() for n,t1 in enumerate(terms) for t2 in terms[:n]]
    return propagate(eqs, goal)
# this isn't eunification. We don't know we've found enough to gurantee the goal is true.
def partial_solve(vs, goal): # try to find solutions to vs in goal
    terms = set(kd.utils.subterms(goal))
    terms.union(vs)
    eqs = [smt.Eq(v, t) for v in vs for t in terms if  v.sort() == t.sort() and kd.utils.free_in(v, t)]
    return propagate(eqs, goal) # but maybe with unfolding?
def solve(vs, goal):
    #eqs = partial_solve(vs, goal) # maybe boosted by the ground equalities. Skolemize first? Maybe open any iniitaial exists quantifiers
    eqs = propagate_eqs(list(kd.utils.subterms(goal)),goal)
    print(eqs)
    #return sympy.solve(eqs, vs)
    return sympy.solve([eq for eq in eqs if eq.arg(0).sort() == smt.RealSort() or eq.arg(0).sort() == smt.IntSort()], vs)
def propagate_consts(terms, goal):
    s = smt.Solver()
    s.add(goal)
    assert s.check() == smt.sat
    m = s.model()
    return propagate([smt.Eq(t, m.eval(t)) for t in terms], goal)
def subst_eqs(eqs, goal):
    if eqs:
        return smt.substitute(goal, *[(eq.arg(0), eq.arg(1)) for eq in eqs])
    else:
        return goal

x,y,z = smt.Reals("x y z")
solve([x,y,z], smt.And(x == 3 + y, y**2 + 3 == 4 + z, smt.And(z == z)))

propagate_consts([x,y], smt.And(x + y == 10, y == x + 2))

def solve(vs, goal):
    eqs = propagate_consts(vs, goal)
    goal = subst_eqs(eqs, goal)
    vs 
    while True:
        eqs_ = partial_solve(vs)
        if eqs_:
            eqs.append(eq_[]0)
            goal = subst_eqs([eq_[0]], goal)
        else:
            return eqs


[And(z == z) ==
And(x == 3 + y, y**2 + 3 == 4 + z, And(z == z)), (z == z) == And(x == 3 + y, y**2 + 3 == 4 + z, And(z == z)), (z == z) == And(z == z), (y**2 + 3 == 4 + z) ==
And(x == 3 + y, y**2 + 3 == 4 + z, And(z == z)), (y**2 + 3 == 4 + z) == And(z == z), (y**2 + 3 == 4 + z) == (z == z), (x == 3 + y) ==
And(x == 3 + y, y**2 + 3 == 4 + z, And(z == z)), (x == 3 + y) == And(z == z), (x == 3 + y) == (z == z), (x == 3 + y) == (y**2 + 3 == 4 + z), z == z, z == z, z == z, y**2 + 3 == 4 + z, y == y, 3 == 3, x == 3 + y]


[x == 4, y == 6]

lift sympy to solvers.
Define the funcdecls in there and copy them in theories.real. Or import * them
inmcude quantifier free stuff from context



In [ ]:

# this isn't eunification. We don't know we've found enough to gurantee the goal is true.
def partial_solve(vs, goal): # try to find solutions to vs in goal
    terms = kd.utils.subterms(goal) + vs
    eqs = [v == t for v in vs for t in terms if kd.utils.free_in(v, t)]
    propagate(eqs, goal) # but maybe with unfolding?


def eunify(vs, goal):
    eqs = partial_eqs(vs, goal)
    s = smt.Solver()
    smt.add(smt.Not(smt.Implies(smt.And(eqs), goal)))
    s.check() == smt.unsat

def sympy_bounce(vs, constr):
    while True:
        # smt.simplify(constrs) also seed these terms?
        eqs = partial_solve(vs, constrs)
        vs = sympy.solve([sympify(eq) for eq in eqs], vs)
        constr = smt.substitute(constr, *vs.items())
        new_eqs = 

def solve(vs, constrs):
    eqs = partial_solve(vs, constrs) # maybe boosted by the ground equalities. Skolemize first? Maybe open any iniitaial exists quantifiers





eunify should
- propagate as much as possible to get rid of vars
- as a last resort, start enumerating models



In [ ]:
def eunify(vs, goal):
    #{v : [t for t in smt.subterms(goal) if kd.utils.free_in(v, t)] for v in vs}
    # + build a model
    maybes = [smt.Eq(v, t) for t in smt.subterms(goal)  for v in vs if t.sort() == v.sort() and kd.utils.free_in(v, t)]
    maybes = propagate(maybes, goal)
    subst = {}
    while maybes:
        maybe = maybes.pop()
        v,t = maybe.children()
        if v not in subst and kd.utils.free_in(v, t): # is it possible for free_in to fail? But it to eventually not fail?
            subst[v] = t
            maybes = [smt.substitute(m, v, t) for m in maybes if not m.arg(0).eq(v)]
    unsolved = [v for v in vs if v not in subst]
    v = unsolved.pop() # branch on v
    goal = smt.substitute(goal, *subst.items())
    # do a fair interleaving rather than
    gens = []
    for t in kd.subterms(goal):
        if t.sort() == v.sort() and kd.utils.free_in(v,t):
            #gens.append(eunify(unsolved, smt.substitute(goal, v, t)))
            for sol in eunify(unsolved, smt.substitute(goal, v, t)):
                yield {**sol, **subst, v : t}
            goal = smt.And(goal, v != t)
    while True:
        s = smt.Solver()
        s.add(goal)
        res = s.check()
        if res == smt.unsat:
            return
        assert res == smt.sat
        m = s.model()
        t = m.eval(v)
        for sol in eunify(unsolved, smt.substitute(goal, v, t)):
            yield {**sol, **subst, v : t}
        goal = smt.And(goal, v != t)

    # branch with each of the possible guess subst.
    # then branch with all remaining ground models.


## solve
What would I need to get sympy like solve capabilities




In [ ]:
def as_poly(e) -> dict[smt.ExprRef, smt.ExprRef]:
    if smt.is_add(e):
        p = as_poly(e.children())
    elif smt.is_mul(e):
        c = e.children()
        coeff, vs = [], []
        for x in c:
            if smt.is_value(x):
                coeff.append(x)
            else:
                vs.append(x)
        p = {smt.simplify(smt.Mul(*vs)) : sum(coeff)}


mono_inv = {
    real.exp : (real.log, exp_log_galois) # forall x y, exp(x) == y -> x == ln(y)
    real.log : (real.exp, log_exp_galois) # forall x y, log(x) == y -> x == exp(y)
    real.sin : (real.asin, sin_asin_galois) # forall x y, -pi <= x <= pi, sin(x) == y -> x == asin(y)

}

# mul, add inversion are special.

def isolate(t):
    if t.decl() == real.exp:
        real.log(t.arg(0))
    elif t.decl() == real.log:
        real.exp(t.arg(0))
    else t.decl() == real.sin:
        return real.asin

def eqbreak(e):
    assert smt.is_eq(e)
    lhs, rhs = e.children()
    if smt.is_add(e):
        p = as_poly(e)
    
def flatten(e): # concat
    ...

def flatten_sort(e): #
    ...



# indexing subsumption

Backward and forward subsumption.
I made a bottom up e-unification routine.

What about a dict that can lookup by equality, or subsumption

keys = {}



Is this an egraph?

models are a great filter.
but for variables... less clear.

range queries

semantic resolution

A \/ B    C \/ D
 ----------------------
A \/ B == C \/ B \/ D


A \/ B \/ C     |= B => A 
 -----------------------
    B \/ C     


Kind of dual of opragate or something.
Must be true, may be true

The lifted truth value model. Model dependent truth.
 True in all models, true is no models, true in some.

No this isn't really what we want.

reduced redunants

Using nominal to implement unification?

Var(a) == Foo. No. This will never unidy.

https://www.philipzucker.com/tele/

clause : list[smt.FuncRef]  over same domain. represents an or.

unify = &. concatebnation of variable lists.


Term Orderings ---> set orderings? They are comptabile with the ground ordering, so every term in the set is larger than every term in other set.


https://people.csail.mit.edu/asolar/SynthesisCourse/TOC.htm

Program synthesis. I have parametrized sets right?

Interval(a,b) = smt.Lambda([x], smt.And(a <= x, x <= b))
Do some minimization to try and 
cegar?






In [ ]:
def synth(params, vs, goal): # cegar, cegis?
    example = smt.Solver()
    counter = smt.Solver()
    while True:
        s.add(smt.substitute(goal, [(v, m.eval(v) for v in vs)]))
        s.check()
        prover = Solver() # This one is not usefully persistent?
        sol = [param == m.eval(param) for param in params]
        prover.add(sol)
        prover.add(smt.Not(goal))
        res = prover.check()
        if res == smt.unsat:
            return sol
        else:
            m = 

In [ ]:
import kdrag.theories.list as list_
from kdrag.all import *
IList = list_.ListSort(smt.IntSort())

x,y = smt.Consts("x y", IList)
# unification pattern as a set
p = smt.Lambda([x], smt.Exists([y], x == IList.Cons(1, y)))

IListPattern = smt.SetSort(IList)

# two patterns can be compared by == variant, <= subsumption.
# a term matches a pattern if p[t]
# pattern are abstract domain like. They represent the set of all terms that could match them. Like interval, they are concrete computabler data that interprets as sets.

kd.prove(p[IList.Cons(1, IList.Nil)])
kd.prove(smt.Not(p[IList.Nil]))

# unification is most general unififier
p & p

# substitution. Why go to bool?
smt.Lambda([y], IList.Cons(1, y))

# equalzier / coequualzier
def Eq(f,g):
    doms = f.domains()
    p = smt.ArraySort(*doms, smt.TupleSort(doms))
    smt.ForAll([vs], f(p(*vs)) == g(p(*vs)))



Lambda(y, Cons(1, y))

In [ ]:
def possibles(goal, maybes):
    s = smt.Solver()
    s.add(goal)
    possible = []
    while True:
        res = s.check()
        if res == smt.unsat:
            return possible
        else:
            assert res == smt.sat
            m = s.model()
            newmaybe = []
            for p in maybes:
                if m.eval(p):
                    possible.append(p)
                else:
                    newmaybe.append(p)
            maybes = newmaybe
            s.add(smt.Or(maybes))



In [ ]:
class Z3Dict():
    # The brute force sweep version of checking semantic equality.
    def __init__(self, merge=None):
        self.data = {}
        if merge is None:
            self.merge = lambda a,b: b
        else:
            self.merge = merge
        self.memo = {}
    def find(self, key): # find key that is always equal
        res = self.memo.get(key)
        if res is not None:
            return res, self.data[res]
        else:
            for k,v in self.data.items():
                s = smt.Solver()
                s.add(k != key)
                if s.check() == smt.unsat:
                    return k,v
            return None
    def get(self, key):
        res = self.find(key)
        if res is None:
            return None
        else:
            return res[1]
    def set(self, key, value):
        res = self.find(key)
        if res is None:
            self.data[key] = value
        else:
            k,v = res
            self.data[k] = self.merge(value, v)
            self.memo[key] = k
    def prune(self, key): # fingerprint. iterator of keys that can match key in model
        for k,v in self.data.items():
            s = smt.Solver()
            s.add(k == key)
            if s.check() == smt.sat:
                yield k,v

            

Some kind of boolean indexing tree might let us narrow down basic on model eval queries
unsat core?
To get down from O(n) queries.

In [ ]:
class Z3Dict:
    def __init__(self):
        self.solver = smt.Solver()
        #self.keyvars = {}
        self.data = {}
        self.models = []
        self.tail = smt.FreshConst(smt.BoolSort(), "tail")
    def __getitem__(self, key : smt.ExprRef):
        v = smt.Const("ket", key.sort())
        with self.solver:
            self.solver.add(v == key)
            self.solver.check()
    def __setitem__(self,)




# logic parse



kd.expr()
kd.define


let x := ...


match
update notation
":" annotation info
let notation

∈
∉
\
Move unpacked in to top level
| x | = absolute value
If I start parsing in C / rust / zig, I get benefits

inductive  where

Things that don't make lemmas should be loaded by default.
split set into helpers vs theorems.


anonymous intros and named intros






type inference - use smt.TypeVar

https://www.engr.mun.ca/~theo/Misc/pratt_parsing.htm
https://eli.thegreenplace.net/2012/08/02/parsing-expressions-by-precedence-climbing
https://en.wikipedia.org/wiki/Operator-precedence_parser


Maybe hard code in BitVec. It's just such a useful one
Maybe I should allow just functions to be called. Also useful. ULE etc
Make smt a default global
Any other weirdo but still ascii stuff?

I could scan the entire system for all FuncDeclRef, ExprRef. I guess that's the same thing.

Or use context.

forall x, foo x   could be inferred if we have foo, but not x in context.




In [ ]:
from kdrag.all import *
kd.expr("fun x : Int => x")
kd.expr("forall x : Int, x + 1 >= x")

kd.ind("inductive BoolType = TrueC | FalseC")


class Context():
    env : dict[str, object]
    def expr(selfm, s : str) -> smt.ExprRef:
        return kd.expr(s, globals=self.env)
    def ind(self, s : str) -> object:
        return kd.ind(s, globals=self.env)
    def Const(name: str, sort: smt.SortRef) -> smt.ConstRef:
        x = smt.Const(name, sort)
        self.env[name] = x
        return x
    def Function(name: str, *domain: list[smt.SortRef], range: smt.SortRef) -> smt.FuncDeclRef:
        f = smt.Function(name, *domain, range)
        self.env[name] = f
        return f
    def add(self, x : smt.ExprRef | smt.SortRef | smt.FuncDeclRef):
        if isinstance(x, smt.ExprRef):
            self.env[x.decl().name()] = x
        elif isinstance(x, smt.SortRef) or isinstance(x, smt.FuncDeclRef):
            self.env[x.name()] = x
    def current(self): ...
        # reify the current context into env
        ## lookup stack




AttributeError: module 'kdrag' has no attribute 'ind'

In [1]:
from kdrag.all import *

@kd.Theorem("forall (x : Int), x + 0 = x")
def add_zero(pf):
    pf.auto()
add_zero

|= ForAll(x, x + 0 == x)

In [70]:
import lark
import inspect
import kdrag.smt as smt
from lark import Tree

grammar = r"""
start: expr

?expr: quantifier
?quantifier: implication | "forall" binders "," expr  -> forall_ | "exists" binders "," expr -> exists_ | "fun" binders "=>" expr -> fun_
?implication: disjunction | disjunction "->" implication  -> implies
?disjunction: conjunction | disjunction ("\\/" | "||") conjunction  -> or_
?conjunction: comparison | conjunction ("/\\" | "&&") comparison  -> and_
?comparison: additive
    | comparison ("=" | "==") additive   -> eq  | comparison "!=" additive  -> neq
    | comparison "<" additive   -> lt  | comparison ">" additive   -> gt
    | comparison "<=" additive  -> le  | comparison ">=" additive  -> ge
?additive: multiplicative
    | additive "+" multiplicative  -> add | additive "-" multiplicative  -> sub
?multiplicative: application
    | multiplicative "*" application  -> mul | multiplicative "/" application  -> div
?application: atom atom* -> app
?atom: const | num | bool_ | "(" expr ")"

binders: binder+
binder: "(" NAME+ ":" sort ")"
?sort: arrow
?arrow: sortatom | sortatom "->" arrow -> array
?sortatom : NAME -> sortlit | "BitVec" NUMBER -> bitvecsort | "(" arrow ")"

const: NAME
num: NUMBER
bool_: "true" | "false"

NAME: /[a-zA-Z_][a-zA-Z0-9_']*/
NUMBER: /-?\d+/
%import common.WS
%ignore WS
COMMENT: "--" /[^\n]*/
%ignore COMMENT
"""

parser = lark.Lark(grammar, start="start", parser="lalr")
parser.parse("x + 1")


def parse(s, globals=None):
    env = {}
    def lookup(name):
        if name in env:
            res = env[name]
        elif globals is not None and name in globals:
            res = globals[name]
        else:
            raise ValueError("Unknown name", name)
        assert isinstance(res, smt.AstRef)
        return res
    def sort(tree):
        match tree:
            case Tree("array", [left, right]):
                return smt.ArraySort(sort(left), sort(right))
                #return smt.ArraySort(*(sort(s) for s in left.children), sort(right))
            case Tree("bitvecsort", [n]):
                return smt.BitVecSort(int(n))
            case Tree("sortlit", [name]):
                if name == "Int":
                    return smt.IntSort()
                elif name == "Real":
                    return smt.RealSort()
                elif name == "Bool":
                    return smt.BoolSort()
                else:
                    s = lookup(name)
                    if isinstance(s, smt.SortRef):
                        return s
                    else:
                        raise ValueError("Name is not a sort", name, s)
            case _:
                raise ValueError("Unknown sort tree", tree)

    def binder(tree):
        match tree:
            case Tree("binder", names_sort):
                names = names_sort[:-1] 
                s = sort(names_sort[-1])
                return [smt.Const(str(name[0]), s) for name in names]
            case _:
                raise ValueError("Unknown binder tree", tree)

    def binders(tree):
        match tree:
            case Tree("binders", bs):
                return [v for b in bs for v in binder(b)]
            case _:
                raise ValueError("Unknown binders tree", tree)
    def quant(vs, body_tree, q):
        nonlocal env
        old_env = env.copy()
        vs = binders(vs)
        for v in vs:
            env[str(v)] = v
        res = q(vs, expr(body_tree))
        env = old_env
        return res
    def expr(tree):
        match tree:
            case Tree("num", [n]):
                return int(n)
            case Tree("const", [name]):
                return lookup(name)
                #return smt.Const(str(name), smt.IntSort()) # default sort int for now
            case Tree("and_", [left, right]):
                return smt.And(expr(left), expr(right))
            case Tree("or_", [left, right]):
                return smt.Or(expr(left), expr(right))
            case Tree("add", [left, right]):
                return expr(left) + expr(right)
            case Tree("sub", [left, right]):
                return expr(left) - expr(right)
            case Tree("mul", [left, right]):
                return expr(left) * expr(right)
            case Tree("div", [left, right]):
                return expr(left) / expr(right)
            case Tree("eq", [left, right]):
                return smt.Eq(expr(left), expr(right))
            case Tree("le", [left, right]):
                return expr(left) <= expr(right)
            case Tree("lt", [left, right]):
                return expr(left) < expr(right)
            case Tree("ge", [left, right]):
                return expr(left) >= expr(right)
            case Tree("gt", [left, right]):
                return expr(left) > expr(right)
            case Tree("app", [func]): # constant
                return expr(func)
            case Tree("app", [func, *args]):
                # auto curry
                args = [expr(arg) for arg in args]
                func = expr(func)
                if isinstance(func, smt.FuncDeclRef):
                    return func(*args)
                elif smt.is_func(func):
                    while args:
                        doms = smt.domains(func)
                        func = func[*args[:len(doms)]]
                        args = args[len(doms):]
                    return func
            case Tree("forall_", [vs, body]):
                return quant(vs, body, smt.ForAll)
            case Tree("exists_", [vs, body]):
                return quant(vs, body, smt.Exists)
            case Tree("fun_", [vs, body]):
                return quant(vs, body, smt.Lambda)
            case Tree("implies", [left, right]):
                return smt.Implies(expr(left), expr(right))
            case _:
                raise ValueError("Unknown parse tree", tree)
    tree = parser.parse(s)
    match tree:
        case Tree("start", [e]):
            return expr(e)
        case _:
            raise ValueError("Invalid parse tree", tree)
parse("x + 1", {"x": smt.Int("x")})
x,y = smt.Ints("x y")
assert parse("forall (x y : Int), x + 1 = 0 -> x = -1").eq(smt.ForAll([x,y], x + 1 == 0, x == -1))
a = smt.Real("a")
assert parse("exists (a : Real), a * a = 2").eq(smt.Exists([a], a * a == 2))
parse("fun (x : Int -> Int) => x 1 = 2")
parse("fun (x : Int -> Int -> Int) => x 1 3 = 2")
parse("f 3 2", {"f": smt.Function("f", smt.IntSort(), smt.IntSort(), smt.IntSort())})
parse("fun (x : Int) (y : Real) => x + y > 0")
parse(r"forall (x : Int), x >= 0 /\ x < 10")
parse(r"forall (x : Int), x >= 0 && x < 10 -> x < 20")
parse(r"exists (x : BitVec 32 -> BitVec 8), x 8 = 5")


Exists(x, x[8] == 5)

In [ ]:
precedence = {
    'or': 1,
    'and': 2,
    'not': 3,
    '==': 4,
    '!=': 4,
    '<': 5,
    '<=': 5,
    '>': 5,
    '>=': 5,
    '+': 6,
    '-': 6,
    '*': 7,
    '/': 7,
    'neg': 8,
}

In [ ]:
import lark
import inspect
import kdrag.smt as smt

grammar = r"""
start: expr

?expr: quantifier
?quantifier: implication | "forall" binders "," expr  -> forall_ | "exists" binders "," expr -> exists_ | "fun" binders "=>" expr -> fun_
?implication: disjunction | disjunction "->" implication  -> implies
?disjunction: conjunction | disjunction ("\\/" | "||") conjunction  -> or_
?conjunction: comparison | conjunction ("/\\" | "&&") comparison  -> and_
?comparison: additive
    | comparison "=" additive   -> eq  | comparison "!=" additive  -> neq
    | comparison "<" additive   -> lt  | comparison ">" additive   -> gt
    | comparison "<=" additive  -> le  | comparison ">=" additive  -> ge
?additive: multiplicative
    | additive "+" multiplicative  -> add | additive "-" multiplicative  -> sub
?multiplicative: application
    | multiplicative "*" application  -> mul | multiplicative "/" application  -> div
?application: atom | application atom  -> app


binders: binder+
binder: "(" NAME+ ":" sort ")"
sort: NAME | sort "->" sort -> arrow

?atom: const | num | bool_ | "(" expr ")"
const: NAME
num: NUMBER
bool_: "true" | "false"

NAME: /[a-zA-Z_][a-zA-Z0-9_']*/
NUMBER: /-?\d+/
%import common.WS
%ignore WS
COMMENT: "--" /[^\n]*/
%ignore COMMENT
"""

parser = lark.Lark(grammar, start="start", parser="lalr")


class T(lark.Transformer):
    def __init__(self, globals=None):
        if globals is None:
            self.globals = {}
        else:
            self.globals = globals
        self.locals = {}

    def _lookup(self, name: str):
        if name in self.locals:
            return self.locals[name]
        elif name in self.globals:
            return self.globals[name]
        else:
            raise ValueError(f"Unknown name: {name}., {self.locals}, {self.globals}")

    def start(self, xs):
        return xs[0]

    def NAME(self, t):
        return str(t)

    def num(self, xs):
        return xs[0]

    def NUMBER(self, t):
        return int(t)

    def const(self, name):
        return self._lookup(name[0])

    def bool_(self, xs):
        return smt.BoolVal(str(xs[0]) == "true")

    def sort(self, xs):
        name = xs[0]
        if name == "Int":
            return smt.IntSort()
        elif name == "Real":
            return smt.RealSort()
        elif name == "Bool":
            return smt.BoolSort()
        else:
            return self._lookup(xs[0])

    def binders(self, xs):
        # Flatten list of binders
        result = []
        for item in xs:
            result.extend(item)
        return result

    def binder(self, xs):
        # xs is [name1, name2, ..., sort]
        *names, sort = xs
        return [(n, sort) for n in names]

    def quant(self, tree, q):
        # Transform binders first
        bs = self._transform_tree(tree.children[0])
        # Add to env
        old_local = self.locals.copy()
        vs = [smt.Const(n, s) for n, s in bs]
        for v, (n, _) in zip(vs, bs):
            self.locals[n] = v
        # Now transform body with vars in scope
        body = self._transform_tree(tree.children[1])
        self.env.locals = old_local
        return q(vs, body)

    @lark.v_args(tree=True)
    def forall_(self, tree):
        return self.quant(tree, smt.ForAll)

    @lark.v_args(tree=True)
    def exists_(self, tree):
        return self.quant(tree, smt.Exists)

    @lark.v_args(tree=True)
    def fun_(self, tree):
        return self.quant(tree, smt.Lambda)

    def app(self, xs):
        # For Z3 functions/lambdas, apply directly
        func, arg = xs[0], xs[1]
        try:
            return func[arg]
        except (TypeError, AttributeError):
            # If not callable, it's likely undefined - return tuple for inspection
            return (func, arg)

    def implies(self, xs):
        return smt.Implies(xs[0], xs[1]) if len(xs) > 1 else xs[0]

    def or_(self, xs):
        return smt.Or(xs[0], xs[1])

    def and_(self, xs):
        return smt.And(xs[0], xs[1])

    def eq(self, xs):
        return xs[0] == xs[1]

    def neq(self, xs):
        return xs[0] != xs[1]

    def lt(self, xs):
        return xs[0] < xs[1]

    def gt(self, xs):
        return xs[0] > xs[1]

    def le(self, xs):
        return xs[0] <= xs[1]

    def ge(self, xs):
        return xs[0] >= xs[1]

    def add(self, xs):
        return xs[0] + xs[1]

    def sub(self, xs):
        return xs[0] - xs[1]

    def mul(self, xs):
        return xs[0] * xs[1]

    def div(self, xs):
        return xs[0] / xs[1]


def parse(s: str, globals=None) -> smt.ExprRef:
    """
    Parse a Lean-like expression to Z3.

    >>> parse("x + 1", globals={"x": smt.Int("x")})
    x + 1
    >>> parse("forall (x : Int), x + 0 = x")
    ForAll(x, x + 0 == x)
    >>> f = smt.Function("f", smt.IntSort(), smt.IntSort())
    >>> parse("f 3", {"f": f})
    f(3)
    """
    return T(globals).transform(parser.parse(s))


def parse(s: str, globals=None) -> smt.ExprRef:
    """
    Parse a Lean-like expression to Z3.

    >>> parse("x + 1", globals={"x": smt.Int("x")})
    x + 1
    >>> parse("forall (x : Int), x + 0 = x")
    ForAll(x, x + 0 == x)
    >>> f = smt.Function("f", smt.IntSort(), smt.IntSort())
    >>> parse("f 3", {"f": f})
    f(3)
    """
    return parser.parse(s)


In [ ]:
import re
from typing import Any
import kdrag.smt as smt
import operator


# Token patterns (order matters for matching)
tokens = [
    ("WS", r"\s+"),
    ("COMMENT", r"--[^\n]*"),
    ("FORALL", r"\bforall\b"),
    ("EXISTS", r"\bexists\b"),
    ("IMPLIES", r"->"),
    ("AND", r"/\\"),
    ("OR", r"\\/"),
    ("NOT", r"~|¬|\bnot\b"),
    ("LPAREN", r"\("),
    ("RPAREN", r"\)"),
    ("LBRACE", r"\{"),
    ("RBRACE", r"\}"),
    ("COMMA", r","),
    ("COLON", r":"),
    ("COMPARE", r"<=|>=|<|>|=|!="),
    ("PLUS", r"\+"),
    ("MINUS", r"-"),
    ("TIMES", r"\*"),
    ("DIV", r"/"),
    ("NUMBER", r"-?\d+(?:\.\d+)?"),
    ("TRUE", r"\bTrue\b"),
    ("FALSE", r"\bFalse\b"),
    ("IDENT", r"[A-Za-z_][A-Za-z0-9_]*"),
]

regex = "|".join(f"(?P<{name}>{pattern})" for name, pattern in tokens)
pattern = re.compile(regex)


def lex(s: str) -> list[re.Match]:
    out = []
    pos = 0
    while pos < len(s):
        m = pattern.match(s, pos)
        if not m:
            raise SyntaxError(f"lex error at {s[pos:pos+20]!r}")
        kind = m.lastgroup
        if kind != "WS":
            out.append(m)
    return out


prec = {
    "MUL": (7, operator.mul),
    "DIV": (7, operator.truediv),
    "PLUS": (6, operator.add),
    "MINUS": (6, operator.sub),
    "COMPARE": (5, None),  # Comparison operators handled separately
    "NOT": (3, operator.not_),
    "AND": (2, operator.and_),
    "OR": (1, operator.or_),
    "IMPLIES": (0, smt.Implies),
    }

class Parser():

    def __init__(self, s):
        self.tokens = lex(s)
        self.pos = 0
    def term(self):
        left = self.atom()
        min_prec = 0
        while True:
            tok = self.tokens[self.pos]
            res = prec.get(tok.lastgroup)

    def expect(self, kind):
        tok = self.tokens[self.pos]
        if tok.lastgroup != kind:
            raise SyntaxError(f"expected {kind}, got {tok.lastgroup} at {tok.start()}")
        self.pos += 1
        return tok
    
    def atom(self):
        tok = self.tokens[self.pos]
        match tok.lastgroup:
            case "LPAREN":
                self.pos += 1
                t = self.expr()
                self.expect("RPAREN")
            case "NUMBER":
                self.pos += 1
                return smt.IntVal(int(tok.group(0)))
            case "TRUE":
                self.pos += 1
                return smt.BoolVal(True)
            case "FALSE":
                self.pos += 1
                return smt.BoolVal(False)
            case "IDENT":
                self.pos += 1
                return smt.Int(tok.group(0))
            case _:
                raise SyntaxError(f"unexpected token {tok.lastgroup} at {tok.start()}")
        



NameError: name 'parser' is not defined

In [ ]:



class Peeker():
    def __init__(self, s):
        self.toks = lex(s)
    def peek(self):
        if not self.queue:
            try:
                self.queue.append(next(self.it))
            except StopIteration:
                pass
        return self.queue[-1] if self.queue else None
    def peekn(self, n=1):
        while len(self.queue) < n:
            try:
                self.queue.append(next(self.it))
            except StopIteration:
                break
        return self.queue[:n]
    def next(self):
        if self.queue:
            return self.queue.pop(0)
        else:
            return next(self.it)
    def eat(self, kind):
        tok = self.next()
        assert tok.lastgroup == kind
        return tok
    def expr(self):
        tok = self.peek()
        match tok.lastgroup:
            case "LPAREN":
                self.next()
                e = self.expr()
                self.eat("RPAREN")
                return e
            case "FORALL":
                return self.forall()
            case "EXISTS":
                return self.exists()
            case "IDENT":
                tok = self.next()

                return smt.Int(tok.group(0)) # only ints for now
    
    def implies(self):
        left = self.expr()
        tok = self.peek()
        if tok and tok.lastgroup == "IMPLIES":
            self.next()
            right = self.expr()
            return smt.Implies(left, right)
        else:
            return left
    
    def or_(self):
        left = self.and_()
        tok = self.peek()
        while tok and tok.lastgroup == "OR":
            self.next()
            right = self.implies()
            left = smt.Or(left, right)
            tok = self.peek()
        return left

    def and_(self):
        left = self.expr()
        tok = self.peek()
        while tok and tok.lastgroup == "AND":
            self.next()
            right = self.or_()
            left = smt.And(left, right)
            tok = self.peek()
        return left

    def 

    def funapp(self):
        tok = self.eat("IDENT")
        return smt.Const(tok.group(0), smt.IntSort()) # only ints for now

    def const(self):
        tok = self.next()
        match tok.lastgroup:
            case "NUMBER":
                if "." in tok.group(0):
                    return smt.RealVal(float(tok.group(0)))
                else:
                    return smt.IntVal(int(tok.group(0)))
            case "TRUE":
                return smt.BoolVal(True)
            case "FALSE":
                return smt.BoolVal(False)
            case _:
                raise ValueError(f"Unexpected token {tok}")
            
    def forall(self):
        tok = self.next()
        assert tok.lastgroup == "FORALL"
        vs = []
        while self.peek().lastgroup != "COMMA":
            vs.append(self.var())
        return smt.ForAll(vs, self.expr())

    def var(self):



p = Peeker(pattern.finditer("forall x: Int, x > 0 -> exists y: Int. y == x + 1"))
p.peek()
p.next()





<re.Match object; span=(0, 6), match='forall'>

# internal union find


# z3 incrmeental mangling
Incrementally manle something to faster form?
https://mastodon.social/@cfbolz/115436203096168412
https://blog.nelhage.com/post/regex-crosswords-z3/

# transitive closure

The built in and just using a goddamn integer
https://github.com/codyroux/traat-lean/blob/main/Traat/chapter1.lean

We need to make an axuliarily evidence holding relation.

I guess to fit the pattern it might be

[R], smt.Lambda([x,y], smt.Lambda([n], ))


In [ ]:
@functools.cache
def ReflTrans(T : smt.SortRef):
    x,y,z = smt.Consts("x y z", T)
    R = smt.Array("R", T, T, smt.BoolSort())
    dt = kd.Inductive("Trans", R, x, y)
    dt.declare("Base", pred=x == y)
    dt.declare("Trans", ("y", T), ("subpf", dt), pred=lambda y, subpf: smt.And(R(x,y), subpf(R,y,z)))
    return dt.create()



In [ ]:
def StarN(T):
    """
    Transitive closureo f a relation
    """
    R = smt.Array("R", T,T, smt.BoolSort())
    n = smt.Int("n")
    x,y = smt.Consts("x y", T)
    starn.define([R], smt.Lambda([n,x,y], smt.If(n <= 0, x == y,
    smt.Exists([y], smt.And(star(n - 1, x, y), R[y, z])
    ))

starn = kd.notation.SortDispatch("starn", default=StarN)
def Star(R):
    return smt.Exists([n], n >= 0, starn(n, x, y))

# named hypotheses
Ok so adding a hyp_name dict got hairy. I want propositions to change structure but them still be applied.
I could just ignore the structure check
hyp_names could go into Goal

have intros return an integer and make it meta.

I don't really clear that often. But split also did stuff, shifted all numbers. COuld change split to put it at the end, not the beginning.






# Defaults in notation

Just put the obvious builtin good stuff into notation so its always available.

Pointwise recursion and const lifting.

Pointwise lifting of boolean operations?
And then `prove` knows to convert pointwise bools to 
That's kind of a radical change.


T -> Bool
Even -> Bool as truth values.
But is it Exists([x], f[x]) or ForAll([x], f[x]) or ForAll([x], Not(finset[x]), ) 
This is presheafy?

make notation change temporary. That's interesting. __enter__ __exit__ push and pop a copy of dict.
a stack of dicts. Could have problems with concurrency later.


kd.Valid()
kd.forall(lam)

bforall



Still I want for example in temporal to register later.


I should move QForAll into smt.ForAll


## Boolean valued models
https://en.wikipedia.org/wiki/Boolean-valued_model
https://plato.stanford.edu/archives/win2019/entries/boolalg-math/


Bell, John (1988). Toposes and Local Set Theories: an Introduction. 
Bell - boolean valued models
mm. Maybe this could be really fun
I could overload almost everything to work on Boolean valued models
I guess the main model of separation logic is a boolean valued model of sets of heaps
I could also bolt in sets of proof objects as a truth values
So the truth is kind of curried here to depend on what universe you’re working in. That kind of makes sense.

https://jdh.hamkins.org/a-gentle-introduction-to-boolean-valued-model-theory/

Make And, Or, Implies overloaded to work with Sets. Ehhh, maybe operator overloading is good enough

Pointwise equality. Interesting that this could matter.

forcing as some kind

In [ ]:
def BEq(x, y):
    if smt.is_func(x):
        doms = domain(x.sort())
        cod = x.sort().range()
        args = [smt.FreshConst(dom) for dom in doms]
        return smt.Lambda(args, BEq( x(*(args)), y(*(args)) ) ) # Map?
    else:
        return smt.Eq(x, y)




In [ ]:
def pointwise(sort, f, *args):
    if smt.is_func(f):
        doms = domain(sort)
        cod = sort.range()
        args = [smt.FreshConst(dom) for dom in doms]
        smt.Lambda(args, self[cod]( *(f(arg) for arg in args) ) )
    self.register(sort, )
    elif smt.SeqSort

    

# subsorting coerce cast

A subsort table

a single super class
castable, coercible

https://rocq-prover.org/doc/V8.10.2/refman/addendum/implicit-coercions.html
https://www.cs.umd.edu/~mwh/papers/coercions.pdf a theory of typed conversions and applications


In [ ]:
subsort_table = {smt.IntSort() : smt.BoolSort(), int : }

coercetable = {(a,b) : , (int, smt.IntSort()) : smt.IntVal}

def coerce(sort, v):
    if isinstance(v, smt.ExprRef):
        if v.sort() == s:
            return v
        else:
            coercetable[(v.sort(), sort)](v)
    else:
        coercetable[(type(v), sort)](v) # try all super types? mro
supersort = {}

# defineable choice

Surpsiirng functionals. Searching infinite space in finite time

choice is sometimes definable.

choose = kd.notation.SortDispatch()

choose over bitvecs is amusing. define a sat solver inside of smt


bin_search
prefix_search

recfunction?

Kind of like that idea of SAT over formula
is_sat(P) == exists

model(x, P) == P[x]






# tla

The default registers pattern would be useful here.


self.prove which can be overloaded in Lemma?
To make custom environments. But then how to get a raw env?

ctx.Lemma could overload this.
explicit is better than implicit


clock
async
fifo



In [ ]:
p = smt.Bool("p")
p <= p
p >= p




NotImplementedError: No implementation of lt for sort Bool. Register a definition via lt.register(Bool, your_code)

In [ ]:
a,b = smt.Bools("a b")
a < b = smt.And(Not(a), b)
a <= b = smt.Implies(a, b)



In [6]:
import kdrag as kd
import kdrag.smt as smt

T = smt.SetSort(smt.IntSort())
a,b = smt.Consts("a b", T)
x = smt.Int("x")
kd.prove(smt.ForAll([x], (a <= b)(x)) == smt.IsSubset(a, b))
#a < b


|= (ForAll(x, Implies(a[x], b[x]))) == subset(a, b)

In [ ]:
from kdrag.all import *
from kdrag.theories.logic.temporal import *

hr = Int("hr")
#hr == If(Not(Eq(hr,TLift(12))), bnext(hr) == hr + TLift(1), bnext(hr) == Const(1))
If(ineq(hr,TLift(12)), hr + TLift(1), TLift(0))
Next(hr) ==If(ineq(hr,TLift(12)), hr + TLift(1), TLift(0))




T_Bool(Lambda(t!167,
              val(T_Int(Lambda(t!165, val(hr)[t!165 + 1])))[t!167] ==
              val(T_Int(Lambda(t!166,
                               If(val(ineq(hr,
                                        T_Int(K(Int, 12))))[t!166],
                                  val(add(hr,
                                        T_Int(K(Int, 1))))[t!166],
                                  val(T_Int(K(Int, 0)))[t!166]))))[t!167]))

In [ ]:

TInt = smt.ArraySort(smt.IntSort(), smt.IntSort())
hr = smt.Const("hr", TInt)
Box(hr <= 4)
t = smt.BitVec("t", 4)
h = smt.Lambda([t], smt.BoolVal(True))
Box(h)



Lambda(x0!130,
       ForAll(x0!131, Implies(And(x0!130 <= x0!131), True)))

In [13]:
HereThere = kd.Enum("HereThere", ["Here", "There"])
n,m = smt.Consts("n m", HereThere)
le = kd.notation.le.register(HereThere, lambda n,m : smt.Not(smt.And(n == HereThere.There, m == HereThere.Here)))
#kd.notation.le.define([n,m], smt.Not(smt.And(n == HereThere.There), m == HereThere.Here))

Box(smt.Lambda([n], smt.BoolVal(True)))


Lambda(x0!119,
       ForAll(x0!120,
              Implies(And(Not(And(x0!119 == There,
                                  x0!120 == Here))),
                      True)))

In [ ]:
x = smt.BitVec("x", 1)
(x <= x) #== smt.ULE(x,x).decl() yeah it's signed, but doesn't matter right?

<=

In [13]:
#from kdrag.all import *
import kdrag as kd
import kdrag.smt as smt
import kdrag.theories.int as int_
TInt = smt.ArraySort(smt.IntSort(), smt.IntSort())
hr = smt.Const("hr", TInt)
hr <= hr + 4
def Next(x):
    t = smt.FreshInt(prefix="t")
    return smt.Lambda([t], x(t+1))
def Always(x, unchanged=None):

    t1 = smt.FreshInt(prefix="t")
    t2 = smt.FreshInt(prefix="t")
    if unchanged is not None:
        return smt.Lambda([t1], smt.ForAll([t2], t2 >= t1, (x | UNCHANGED(*unchanged)(t2))))
    else:
        return smt.Lambda([t1], smt.ForAll([t2], t2 >= t1, x(t2)))
def Valid(x):
    return x(0)
def UNCHANGED(*args):
    t = smt.FreshInt(prefix="t")
    return smt.Lambda([t], smt.And(*(arg(t+1) == arg(t) for arg in args)))

PEq = kd.notation.PEq
PIf = kd.notation.PIf

def TImplies(a, b):
    t = smt.FreshInt(prefix="t")
    return smt.Lambda([t], smt.Implies(a(t), b(t)))



#l = kd.Lemma(Valid(TImplies(HCini & Always(HCtrans), Always(HCini))))

HCini = (0 <= hr) & (hr <= 12)
HCtrans = PEq(Next(hr), PIf(hr < 12, hr + 1, 0)) # todo stutter

@kd.Theorem(Valid(TImplies(HCini & Always(HCtrans), Always(HCini))))
def clock_inv(l):
    l.intros()
    _t = l.fix()
    l.induct(_t)
    l.auto() # neg t
    l.auto() # 0
    l.auto() # pos t
clock_inv

UNCHANGED(hr, hr)
Always(HCtrans,unchanged=[hr])

Z3Exception: Value cannot be converted into a Z3 Boolean value

# measure and termination
Make measure definition for inductive a part of kernel
Structural check
measure = kd.notation.SortDispatch("measure", )


For each recursive call, the absolution value of measure must be <= 0.
smt.And(measure(new_call0) <= measure(orig))

at least one must be decreasing
smt.Abs(smt.Sum()) <= smt.Abs(sum(args))

raw define in kernel
kd.define in tactics




# claude code

updated claude.md. Had it try to fill it out in a manner to explain what it sees.

renaming instan -> specialize. a good suggesiton. i am not trying to break with eisting conventions for no reason

intlist.py. Did good. i may basically have these proofs in there. Still, nice.


Generic Tuple facilities might be nice
I'm not sure if i should deprecate >> for functions types


Have mathlib and isabelle available to search in a subfolder.

try fundamental theorem of arithmetic

It isn't using unfold much or at all


Should I be using api credits
how to retain sessions?


I should enforce aving different names for the arguents of inductives
Or just keeping them implicitly derived more often.

Make primrec that used tuple patterns instead of kwargs?



# verilog

Challenge problem:
barrel shift multiplier performs multiplication
https://zipcpu.com/zipcpu/2021/07/03/slowmpy.html

In [52]:
from kdrag.all import *

W = 64
x, y, z = smt.BitVecs("x y z", W)

#kd.prove(x*y == smt.Sum(smt.If(y & smt.BitVecVal(1 << i, W) == 0, 0, (x << i)) for i in range(W)), timeout=10000)

kd.prove((x + y) * z == x*z + y*z)
kd.prove(x == smt.Concat(smt.BitVecVal(0, W//2), smt.Extract(W//2-1, 0, x)) + smt.Concat(smt.Extract(W-1, W//2, x), smt.BitVecVal(0, W//2)))
kd.prove(x == smt.Concat(smt.BitVecVal(0, W//2), smt.Extract(W//2-1, 0, x)) | smt.Concat(smt.Extract(W-1, W//2, x), smt.BitVecVal(0, W//2)))
one = smt.BitVecVal(1,W)
kd.prove(x == smt.Sum(x & (one << i) for i in range(W)))
lemma = kd.prove(y == smt.Sum(smt.If(y & smt.BitVecVal(1 << i, W) == 0, 0, one << i) for i in range(W)))

"""
@kd.PTheorem(x*y == smt.Sum(smt.If(y & smt.BitVecVal(1 << i, W) == 0, 0, (x << i)) for i in range(W)))
def barrel(l):
    l.rw(lemma)
    assoc = kd.prove(smt.ForAll([y,z], x * (y + z) == x*y + x*z))
    l.rw(assoc)
    l.rw(assoc)
    #l.auto()
"""

def lemma(W):
    x, y = smt.BitVecs("x y", W)
    one = smt.BitVecVal(1,W)
    return kd.prove(y == smt.Sum(smt.If(y & smt.BitVecVal(1 << i, W) == 0, 0, one << i) for i in range(W)))

def divide(W):
    if W == 1:

    x = smt.BitVec("x", W)
    kd.prove(smt.ForAll([x], x == smt.Concat(smt.BitVecVal(0, W//2), smt.Extract(W//2-1, 0, x)) + smt.Concat(smt.Extract(W-1, W//2, x), smt.BitVecVal(0, W//2))))

divide(8)







NameError: name 'divid' is not defined

# universe


In [ ]:
from kdrag.all import *
Base = kd.InductiveType("Base")
Base.declare("Int", ("int", smt.IntSort()))
Base.declare("Bool", ("bool", smt.BoolSort()))
Base.declare("Real", ("real", smt.RealSort()))
Base = Base.create()

Univ = smt.DatatypeSort("Univ")
Univ.declare("Base", ("b", Base))
Univ.declare("Array", ("array", smt.ArraySort(Base, Univ)))
Univ.declare("Seq", ("elem", smt.SeqSort(Univ)))
Univ.create()


BaseType = kd.Enum("BaseType", ["Int", "Bool", "Real"])
Type = kd.InductiveType("Type")
Type.declare("Base", ("basetype", BaseType))
Type.declare("Array", ("dom", BaseType), ("cod", Type))
Type.declare("Seq", ("elem", Type))





AttributeError: module 'kdrag' has no attribute 'InductiveType'

What about an ndarray


# ndarray
conv(n, v, w)
dot(n, v, w) = dots centered around 
conv ~ dot

The shaep is part of the operator, not the data.
Hmm.

matmul(a, n, b)

https://www.cs.dartmouth.edu/~doug/powser.html powser using

I've decided that Nat -> R  is better than ~ equiv to stream.
So all thgose coniductive ideas about power series should be trasnferred to this.





In [ ]:
smt.SeqSort(smt.IntSort())
ind_shape = kd.define("indshape", [N,i],)

sum(N, x) = cumsum(x)[N]
dot(N, x, y) = sum(N, x * y)




Order Big O https://terrytao.wordpress.com/2025/05/09/a-tool-to-verify-estimates-ii-a-flexible-proof-assistant/

c << d
c >> d

trasnitive relationship.
https://o-forge.com/
https://github.com/ayushkhaitanrutgers/decompositions-2
https://en.wikipedia.org/wiki/Big_O_notation

p == q + O(r)  ---> abs(p - q) << r

Is this an equation really?

eventually_le
eventually_
dominates = kd.define("dominates", [a,b], smt.Exists([N], smt.QForAll([n], n > N, a[n] <= b[n])))
dominates = kd.define("dominates", [a,b], smt.Exists([C, N], C > 0, smt.QForAll([n], n > N, a[n] <= C*b[n])))

Kind of the frist is the existentiall opened version of the second, since b will just need to include an explicit calculation of C



# fundamental theorme of arith


# fast


In [10]:
import z3
x = z3.Int('x')
decl = (x + x).decl()
z = x + x
ctx = z3.main_ctx()

In [20]:
%%timeit
decl(x, x)

24.2 μs ± 320 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [21]:
%%timeit
x + x

22.6 μs ± 360 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [26]:
z3.main_ctx().ctx

In [28]:
x.as_ast()

In [ ]:
#z3.Z3_mk_app(decl.ref)


In [7]:
def mk_bin(f, a, b):
    args = (z3.Ast * 2)()
    args[0] = a.as_ast()
    args[1] = b.as_ast()
    return f(a.ctx.ref(), 2, args)

In [23]:
z3.z3_debug()

False

In [25]:
%%timeit
decl(x,x)

21.6 μs ± 179 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
z3.z3.Z3_DEBUG = False
#z3.Z3_DEBUG

In [11]:
%%timeit
z3.ArithRef(mk_bin(z3.Z3_mk_add, x, x), ctx)

2.25 μs ± 17.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [24]:
%%timeit
x + x

19.7 μs ± 50.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [32]:
(z3.Ast * 2)()

In [50]:
z = x + x

In [4]:
%%timeit
z.arg(1)

5.46 μs ± 28.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [5]:
%%timeit
z.eq(x)

691 ns ± 3.49 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [8]:
%%timeit
z3.Z3_mk_add(z3.main_ctx().ctx, 2, x.as_ast(), x.as_ast())


AttributeError: 'Ast' object has no attribute 'f'

In [49]:
%%prun -s tottime
for _ in range(10000):
    decl(x, x)

         3040477 function calls (3010515 primitive calls) in 0.718 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   310000    0.106    0.000    0.118    0.000 z3core.py:1576(Check)
   120000    0.082    0.000    0.139    0.000 z3core.py:2873(Z3_sort_to_ast)
   9998/0    0.037    0.000    0.000          z3.py:855(__call__)
    50000    0.034    0.000    0.058    0.000 z3core.py:1646(Z3_inc_ref)
   210000    0.034    0.000    0.044    0.000 z3.py:410(ctx_ref)
   120000    0.034    0.000    0.196    0.000 z3.py:576(as_ast)
    50000    0.033    0.000    0.136    0.000 z3.py:360(__del__)
    50000    0.032    0.000    0.163    0.000 z3.py:355(__init__)
   670000    0.028    0.000    0.028    0.000 z3types.py:24(from_param)
    50000    0.028    0.000    0.032    0.000 z3core.py:1650(Z3_dec_ref)
    40000    0.026    0.000    0.045    0.000 z3core.py:3118(Z3_get_ast_id)
    30000    0.021    0.000    0.035    0.000 z3core.py:3

In [28]:
Foo = z3.DeclareSort("Foo")
bar = z3.Function("bar", Foo, Foo)
biz = z3.Const("biz", Foo)

In [30]:
%%timeit
bar(biz)

8.65 μs ± 49 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


It makes sorts way too many times.
_coerce


In [31]:
%%timeit 
x = 7
if x == 0:
    y = 4
elif x == 1:
    y = 5
elif x == 2:
    y = 6
elif x == 3:
    y = 7
elif x == 4:
    y = 8
elif x == 5:
    y = 9
elif x == 6:
    y = 10
elif x == 7:    
    y = 11
elif x == 8:
    y = 12

42.6 ns ± 0.21 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [29]:
%%prun -s cumtime
for _ in range(10000):
    bar(biz)

         1110682 function calls (1090670 primitive calls) in 0.277 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.277    0.277 {built-in method builtins.exec}
        1    0.005    0.005    0.277    0.277 <string>:1(<module>)
    10000    0.022    0.000    0.261    0.000 z3.py:855(__call__)
    10000    0.005    0.000    0.084    0.000 z3.py:792(domain)
    10000    0.012    0.000    0.069    0.000 z3.py:1176(_to_expr_ref)
    10000    0.007    0.000    0.061    0.000 z3.py:674(_to_sort_ref)
    20000    0.014    0.000    0.058    0.000 z3.py:355(__init__)
    20000    0.014    0.000    0.047    0.000 z3.py:360(__del__)
   100000    0.035    0.000    0.039    0.000 z3core.py:1576(Check)
    20000    0.006    0.000    0.035    0.000 z3.py:576(as_ast)
    10000    0.008    0.000    0.027    0.000 z3.py:152(_get_args)
    20000    0.014    0.000    0.024    0.000 z3core.py:2873(Z3_sort_to_as

In [27]:
%%prun -s cumtime
for _ in range(10000):
    x + x

         2810560 function calls (2810554 primitive calls) in 0.656 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      2/1    0.000    0.000    0.656    0.656 {built-in method builtins.exec}
      2/1    0.004    0.002    0.652    0.652 <string>:1(<module>)
    10000    0.009    0.000    0.637    0.000 z3.py:2499(__add__)
    10000    0.018    0.000    0.547    0.000 z3.py:1245(_coerce_exprs)
    20000    0.012    0.000    0.244    0.000 z3.py:2407(cast)
   140000    0.040    0.000    0.233    0.000 z3.py:576(as_ast)
    40000    0.027    0.000    0.233    0.000 z3.py:2464(sort)
    20000    0.008    0.000    0.184    0.000 z3.py:1226(_coerce_expr_merge)
    50000    0.034    0.000    0.168    0.000 z3.py:355(__init__)
    30000    0.022    0.000    0.167    0.000 z3.py:414(eq)
   140000    0.096    0.000    0.165    0.000 z3core.py:2873(Z3_sort_to_ast)
    50000    0.034    0.000    0.141    0.000 z3.py:360(__del__)
  

# effect handlers
Hmm.
do(Read, v)
do(Write, a)

How to keep the continuation straigth?

bad idea prob: use sexp.



# llvmlit jit
Talk 
https://github.com/sdiehl/numpile/blob/master/numpile.py

Sortdispatch reshitry of sort to compilation
singledispatch registry of 

emit mlir also




In [15]:
from llvmlite import ir
import llvmlite.binding as llvm
int_type    = ir.IntType(32)

#def of 


def to_llvm(f : smt.FuncDeclRef):
    ir.Function(module, func_type, name)


In [20]:
llvm.initialize_native_target()
llvm.initialize_native_asmprinter()  # yes, even this one
target = llvm.Target.from_default_triple()
target_machine = target.create_target_machine()
backing_mod = llvm.parse_assembly("")
engine = llvm.create_mcjit_compiler(backing_mod, target_machine)

mod = ir.Module()
mod = llvm.parse_assembly(str(module))
mod.verify()

pmb = llvm.PassManagerBuilder()
pmb.opt_level=3
pmb.loop_vectorize = True

pm = llvm.ModulePassManager()

NameError: name 'module' is not defined

In [ ]:
def to_lltype(sort : smt.SortRef):
    if sort == smt.BoolSort():
        return ir.IntType(1)
    elif isinstance(sort, smt.BitVecSortRef):
        return ir.IntType(sort.size())
    

def compile(defn):
    fn_ty = ir.FunctionType(to_lltype(defn.body.sort()), [to_lltype(arg.sort()) for arg in args])
    fn = ir.Function(mod, fn_ty, name=defn.name)
    env = {arg : fn.args[n] for n, arg in enumerate(defn.args)}
    block = fn.append_basic_block("entry")
    b = ir.IRBuilder(block)
    res = compile_expr(b, env, defn.body)
    b.ret(res)


def of_decl(f : smt.FuncDeclRef) -> :
    fn_ty = ir.FunctionType(to_lltype(f.range()), [to_lltype(arg) for arg in f.domain()])


def compile_expr(builder, env, e : smt.ExprRef):
    if e in env:
        return env[e]
    else:
        if smt.is_app(e):

            builder.call(of_decl(e.decl()) , tail="musttail")
        if smt.is_add(e):
        elif smt.is_ite(e):
            builder.select(compile_expr(builder, env, e.arg(0)),
                            compile_expr(builder, env, e.arg(1)),
                            compile_expr(builder, env, e.arg(2)))
        elif smt.is_value(e):
            if smt.is_bool(e):
                return ir.Constant(ir.IntType(1), 1 if e.as_bool() else 0)
            elif smt.is_bv(e):
                return ir.Constant(ir.IntType(e.size()), e.as_long())
            else:
                raise NotImplementedError(f"no constant for {e}")





    

In [27]:
from llvmlite import ir, binding
import ctypes, sys

# 1) Expose CPython symbols to the JIT
pythonapi = ctypes.pythonapi
PyLong_FromLong = pythonapi.PyLong_FromLong
PyLong_FromLong.restype = ctypes.py_object  # convenient boxing for demo
PyLong_FromLong.argtypes = [ctypes.c_long]

#binding.initialize()
binding.initialize_native_target()
binding.initialize_native_asmprinter()
binding.add_symbol("PyLong_FromLong", ctypes.cast(PyLong_FromLong, ctypes.c_void_p).value)

# 2) Build a tiny module that calls PyLong_FromLong(long) -> PyObject*
mod = ir.Module()
i64 = ir.IntType(64)
voidptr = ir.IntType(8).as_pointer()  # stand-in for PyObject*

fn_ty = ir.FunctionType(voidptr, [i64])
fn = ir.Function(mod, fn_ty, name="make_int")

callee_ty = ir.FunctionType(voidptr, [i64])
callee = ir.Function(mod, callee_ty, name="PyLong_FromLong")

block = fn.append_basic_block("entry")
b = ir.IRBuilder(block)
res = b.add(fn.args[0], ir.Constant(i64, 1))
res = b.call(callee, [res])

b.ret(res)

llvm_ir = str(mod)

# 3) JIT and run
target = binding.Target.from_default_triple()
tm = target.create_target_machine()
backing_mod = binding.parse_assembly(llvm_ir)
backing_mod.verify()
engine = binding.create_mcjit_compiler(backing_mod, tm)
engine.finalize_object()

addr = engine.get_function_address("make_int")
MakeInt = ctypes.CFUNCTYPE(ctypes.py_object, ctypes.c_long)(addr)

# GIL is held in Python here, so we can return a real PyObject safely:
obj = MakeInt(123)
print(obj, type(obj))  # -> 123 <class 'int'>


124 <class 'int'>


# Instability

Proofsmith - Michigan. Can mutate proofs to figure out what triggers you should have, try to simplify proofs. Try to be self improving. Dafny


relevancy propagation is an improtant parameter?
CVC5 sumit
smt vs sat solver.
Tactic choices

I should collect my instability exmaples

Triggers. I haven't been using them. Everything else smt based is all about them?

simp.add()  should make lhs a trigger also.

Tpot
SMT25 a conjecture

Can Cebeci disganosiung instability




In [ ]:
def trigger(pf : kd.Proof, trigger : smt.Pattern) -> kd.Proof:
    assert isinstance(pf, kd.Proof)
    thm = pf.thm
    assert isinstance(thm, smt.QuantifierRef) and thm.is_forall()
    # something.

# replace trigger? Have defn trigger by default on lhs.



# contextvars
https://docs.python.org/3/library/contextvars.html

Register a file local context, or explciitly hand it to Lemma or Goal


ctx = kd.Ctx()

x = ctx.add_freshvar(smt.FreshVar())
x = ctx.FreshVar()
  = ctx.FreshVars()


ctx.Goal()

ctx.Lemma()

Locale()


class Lemma
    bigctx



What a weird api.
You can use this to do implicit returns also


In [31]:
import contextvars

ctx = contextvars.Context()
v1 = contextvars.ContextVar("foo")
list(ctx.keys())

v1.set(10)
print(v1.get())
ctx = contextvars.copy_context()

@ctx.run
def _():
    print(list(contextvars.copy_context().items()))
    print(v1.get())
    v1.set(42)
    print(contextvars.copy_context()[v1])
    print(ctx.get(v1))
    assert v1.get() == 42

print("get", v1.get())
list(ctx.keys())
{
    "foo": v1
}

10
[(<ContextVar name='foo' at 0x7d96463c0b80>, 10), (<ContextVar name='foo' at 0x7d96464e2980>, 10), (<ContextVar name='foo' at 0x7d964662a9d0>, 10), (<ContextVar name='foo' at 0x7d9646649350>, 10), (<ContextVar name='foo' at 0x7d964651d350>, 10), (<ContextVar name='foo' at 0x7d96463c6750>, 10), (<ContextVar name='foo' at 0x7d96463c0d60>, 10), (<ContextVar name='foo' at 0x7d9646381a80>, 10), (<ContextVar name='foo' at 0x7d964646b4c0>, 10), (<ContextVar name='foo' at 0x7d96463c04f0>, 10), (<ContextVar name='parent_header' at 0x7d9668c97e70>, {'date': datetime.datetime(2025, 10, 2, 22, 11, 19, 20000, tzinfo=tzutc()), 'msg_id': '42165bb4-4357-4931-8f6e-aec77afd02a0', 'msg_type': 'execute_request', 'session': '2e88a59e-ef1e-402b-bc45-caa34516e0f0', 'username': '194a7128-5aa1-46bf-b35f-239ed23844a2', 'version': '5.2'}), (<ContextVar name='foo' at 0x7d96464e3a10>, 10), (<ContextVar name='foo' at 0x7d96463c4220>, 10), (<ContextVar name='foo' at 0x7d9668c94c20>, 10), (<ContextVar name='foo' a

{'foo': <ContextVar name='foo' at 0x7d964662a9d0>}

# assumes


ExprRef.assumes = None
Optional[BoolRef]
ExprRef.asserts = None
Optional[BoolRef | kd.Proof]

x.assumes = 

kd.QForAll detects assumes. Very similar to wf mechanism but not type based.

vs 
tele system

pdefine(  ,vs, )
regular define could look at this stuff. But what about unfold?
Yea, the if then else -> undef strategy would work

QLambda - a tactic that seeks all if then elses and cases on them

case_all


In [1]:
from kdrag.all import *
x = smt.Int("x")
x.assumes = x >= 0
kd.QForAll([x], x > 1)

ForAll(x, Implies(x >= 0, x > 1))

In [ ]:
from kdrag.rewrite import *
def rename_vars2(
    pf: kd.Proof, vs: list[smt.ExprRef], patterns=[], no_patterns=[]
) -> kd.Proof:
    """
    Rename bound variables and also attach triggers

    >>> x,y = smt.Ints("x y")
    >>> f = smt.Function("f", smt.IntSort(), smt.IntSort())
    >>> rename_vars2(kd.prove(smt.ForAll([x, y], x + 1 > x)), [y,x], patterns=[x+y])
    |= ForAll([y, x], y + 1 > y)
    >>> rename_vars2(kd.prove(smt.Exists([x], x + 1 > y)), [y])
    Traceback (most recent call last):
        ...
    ValueError: ('Cannot rename vars to ones that already occur in term', [y], Exists(x, x + 1 > y))
    """
    assert isinstance(pf, kd.Proof)
    thm = pf.thm
    assert isinstance(thm, smt.QuantifierRef)
    t_body = thm.body()
    body = smt.substitute_vars(t_body, *reversed(vs))
    if thm.is_forall():
        t2 = smt.ForAll(vs, body, patterns=patterns, no_patterns=no_patterns)
    elif thm.is_exists():
        t2 = smt.Exists(vs, body, patterns=patterns, no_patterns=no_patterns)
    else:
        raise Exception("Unknown quantifier type", thm)
    if not t2.body().eq(t_body):
        raise ValueError(
            "Cannot rename vars to ones that already occur in term", vs, thm
        )
    return kd.axiom(t2, by=["rename", pf, vs])

class HintDB(list):
    def add_rw(self, rw : kd.Proof, rev=False):
        # find lhs. Turn into pattern
        rule = rewrite_of_expr(rw)
        if rule.vs:
            if not rev:
                self.append(rename_vars2(rw, rule.vs, patterns=[rule.lhs]))
            else:
                self.append(rename_vars2(rw, rule.vs, patterns=[rule.rhs]))
        else:
            self.append(rw)
    def add_horn(self, horn : kd.Proof):
        rule = rule_of_expr(horn)
        if rule.vs:
            self.append(rename_vars2(horn, rule.vs, patterns=[rule.conc]))
        else:
            self.append(horn)
    def prove(self, goal : smt.ExprRef, by=[]) -> kd.Proof:
        # find relevant rewrites
        # find relevant horn clauses
        # do a search
        return kd.prove(goal, by=self + by)
    def __or__(self, other):
        new = AutoDB(self)
        new.update(other)
        return new

import kdrag.theories.logic.zf as zf

hint = HintDB()
hint.add_horn(zf.elem_emp)
hint.add_rw(zf.elem_upair)
hint.add_rw(zf.elem_sep)
hint.add_rw(zf.elem_union)
hint.add_rw(zf.elem_inter)
hint.add_rw(zf.elem_sing)
hint.add_rw(zf.elem_pair)

hint.prove(zf.elem(zf.emp, zf.sing(zf.emp)))
hint.prove(zf.elem(zf.emp, zf.sing(zf.emp) | zf.emp))
hint.prove(zf.elem(zf.emp, zf.sing(zf.emp) | zf.emp))





|= elem(emp, union(sing(emp), emp))

In [8]:
from kdrag.all import *
x = smt.Int("x")
smt.ForAll([x], x > 0, patterns=[x > 0]).pattern(0)

Var(0) > 0

In [ ]:
def qfixes():
    # can't really tell what constraint belongs to what.
    # we'd need a whole other system to talk about variable constraints.
    

It's kind of like 
even() has a diffgerent truith value. Sets of evidence is the truth value.


In [14]:
from kdrag.all import *


n,m = smt.Ints("n m")
RVec = kd.InductiveRel("Vec", n)
RVec.declare("Nil", pred = n == 0)
RVec.declare("Cons", ("hd", smt.RealSort()), ("tl", RVec), pred=lambda h,t: smt.Exists([m], smt.And(n > 0, t(m), n == m + 1)))
RVec = RVec.create()

kd.prove(RVec(0)[RVec.Nil], unfold=1)
kd.prove(RVec(1)[RVec.Cons(6, RVec.Nil)], unfold=2)
#kd.datatype.call_dict

v,w = smt.Consts("v w", RVec)
n,m = kd.FreshVars("n m", smt.IntSort())

# I kind of can have implicits

v.assumes = RVec(n)[v]
w.assumes = RVec(m)[w]

kd.QForAll([v,w], v == w)


ForAll([v, w],
       Implies(And(vec(n!132)[v], vec(m!133)[w]), v == w))

In [8]:
ISet = smt.SetSort(smt.IntSort())
s,s1 = smt.Consts("s s1", ISet)
FinSet = kd.InductiveRel("FinSet", s)
FinSet.declare("Empty", pred = s == smt.EmptySet(smt.IntSort()))
FinSet.declare("Insert", ("x", smt.IntSort()), ("pred", FinSet), pred = lambda x,p: kd.QExists([s1], p(s1), s == smt.Store(s1, x, True)))
FinSet = FinSet.create()

finev = smt.Const("finev", FinSet)
FinSet(s)
finev(s)


finset(s)[finev]

# Locales

So locales are like named contexts we can be in?
That is different from named collections of truths, but related.
Locales aren't proofs. They're like a second big context to be in?

ctx = Locale()
ctx

https://proofassistants.stackexchange.com/questions/23/what-are-locales-in-isabelle
https://lawrencecpaulson.github.io/2022/03/23/Locales.html locales are predicates?

group()


Maybe locale/ctx could auto filter what's relevant to a particular

Monomorphic locales
https://isabelle.in.tum.de/doc/locales.pdf

In [6]:
from kdrag.all import *
T = smt.DeclareSort("T")
e = smt.Const("e", T)
mul = smt.Function("mul", T, T, T)
x,y,z = smt.Consts("x y z", T)
kd.notation.mul.register(T, mul)
monoid = kd.define("monoid", [], 
smt.And(
    smt.ForAll([x,y,z], (x * y) * z == x * (y * z)),
    smt.ForAll([x], e * x == x),
    smt.ForAll([x], x * e == x)
))
assoc = kd.prove(kd.QForAll([x,y,z], monoid, (x * y) * z == x * (y * z)), unfold=1)
left_id = kd.prove(kd.QForAll([x], monoid, e * x == x), unfold=1)
right_id = kd.prove(kd.QForAll([x], monoid, x * e == x), unfold=1)

class Monoid():
    # requires no proof obligation. As opposed to passing in all proofs, or bys
    def __init__(self, T, e, mul):
        self.T = T
        self.mul = mul
        self.e = e
        self.monoid = kd.define(f"monoid_{mul.decl().name()}", [],
        smt.And(
            smt.ForAll([x,y,z], (x * y) * z == x * (y * z)),
            smt.ForAll([x], e * x == x),
            smt.ForAll([x], x * e == x)
        ))
        self.monoid = kd.define(f"monoid_{mul.decl().name()}", [mul, e], # using Array and constant inputs.
        smt.And(
            smt.ForAll([x,y,z], (x * y) * z == x * (y * z)),
            smt.ForAll([x], e * x == x),
            smt.ForAll([x], x * e == x)
        ))
        self.assoc = kd.prove(kd.QForAll([x,y,z], monoid, (x * y) * z == x * (y * z)), unfold=1)
        self.left_id = kd.prove(kd.QForAll([x], monoid, e * x == x), unfold=1)
        self.right_id = kd.prove(kd.QForAll([x], monoid, x * e == x), unfold=1)






assoc

class Locale():
    def __init__(self, name):
        self.name = name
        self.defns = {}
        self.assumptions = {}
    def assumes(self, name, f):
    def fix(self, ): 
    def create(self, *params):
        ctx = object()
        setattr(ctx, self.name
        kd.define(self.name, [], smt.And(
            [a(*params) for a in self.assumptions.values()]
        )))
        for assumption.values():
            kd.prove(kd.QForAll)




|= ForAll([x, y, z],
       Implies(monoid,
               mul(mul(x, y), z) == mul(x, mul(y, z))))

In [ ]:
@dataclass
class Context():
    fixes : list[smt.ExpRef]
    assumes : list[smt.ExpRef]

    def obtains(self, n): #?

def obtains(vs, fm):
    


In [ ]:
A = smt.DeclareTypeVar("A")
x,y,z = smt.Consts("x y z", A)
mul = smt.Function("mul", A, A, A)
e = smt.Const("e", A)

group = kd.define("group", [], 
    smt.And(
        smt.ForAll([x,y,z], mul(mul(x,y), z) == mul(x, mul(y,z))),
        smt.ForAll([x], mul(x, e) == x),
        smt.ForAll([x], mul(e, x) == x),
    )
)

a = smt.Int("a")
kd.rewrite.unfold(group)

def Group(s : smt.SortRef, op=None, e=None):
    if op is None:
        op = smt.Function("mul", s, s, s)
    elif e is None:
        e = smt.Const("e", s)
    return smt.And(
        smt.ForAll([x,y,z], op(op(x,y), z) == op(x, op(y,z))),
        smt.ForAll([x], op(x, e) == x),
        smt.ForAll([x], op(e, x) == x),
    )


# SortDispatch with default definer?

l = kd.Lemma(smt.Implies(group(S), mul(a, e) == a))
l.intros()
l.unfold(group, at=1)



# unsound


If I switched to tracking an id set,
why even have FreshVar judgement anymore?
Why have a Proof judgement? I could just track ids of proven theorems?


Is the herb form also unsound?




In [7]:
from kdrag.all import *
# Bascially I can't have the ability to skolem and universal var because they enable me to swap exists and forall.
N = kd.FreshVar("N", smt.IntSort())
x = smt.Const("x", smt.IntSort())
pf1 = kd.prove(smt.Exists([x], x > N))
xs, pf2 = kd.tactics.skolem(pf1)
pf3 = pf2.forall([N])(xs[0] + 1)
pf3
kd.prove(smt.BoolVal(False), by=[pf3])

LemmaError: (False, 'Countermodel', [N!23 = 3, f!26 = [1 -> 2, else -> 0]])

In [1]:
frozenset()

frozenset()

In [5]:
EMPTY = frozenset()
def build():
    return frozenset()
def refer():
    return EMPTY

Definitions need to be ALLVAR, but having a var in the definition is a big nono.


In [ ]:
%%timeit

x = frozenset()

27.3 ns ± 0.119 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [4]:
%%timeit
x = EMPTY

7.32 ns ± 0.0199 ns per loop (mean ± std. dev. of 7 runs, 100,000,000 loops each)


In [6]:
%%timeit
build()

53.5 ns ± 0.903 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [7]:
%%timeit
refer()

18.7 ns ± 0.112 ns per loop (mean ± std. dev. of 7 runs, 100,000,000 loops each)


In [8]:
%%timeit
EMPTY | EMPTY

31.8 ns ± 0.204 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [9]:
%%timeit
EMPTY if len(EMPTY) == 0 else EMPTY | EMPTY

17.7 ns ± 0.0467 ns per loop (mean ± std. dev. of 7 runs, 100,000,000 loops each)


A formula has an implicit opening quantifier chain
exists constants, forall freshvars

A formula can always be weakened to 
exists constants, forall fvs <= freshvars

The implicit prefix grows with time (?) Or rather, because the freshvariables don't exist yet, also any formula can be strengthened to placing forall futurevars anyhwere
In particular
exists constants, forall freshvars, forall futurevars

Everything is considered auto strengthened to the currentdepth.
Every formula stands for a set of formulas between the qdepth and currentdepth of quantifiers.

{ exists cs, exists futurecs, forall freshvars,  forall futurevs, forall futurevars in possiblevars}

An existenail proof to be skolemized needs to have a combination of weakening and pulling through applied
exists cs, forall freshvars, exists vs =weaken> exists cs, forall fvs, exists vs  =pull>  exists cs, vs(fvs) forall fvs, 

exists cs, forall freshvars, exists vs =future_strengthen> exists cs, forall freshvars, forall futurevars, exists vs =weaken> 


Every Proof is actually an abstract domain of non implicit quantifier proofs.
This is because I don't want to have to reprove or carryu massive sets of all fresh variables around.


Making the weakening choice at the skolem point is nicer than making it at the

Definitions are kind of skolemizations of   exists t, t = body, fixes=args  and then apply generalization to args.
So definition axioms should get the currentdepth. There really shouldn't be variables in their body.

unfold 

https://arxiv.org/abs/2010.07516 julia world age
macro phase level
"isn't what you're doing macro programming" yes...
"global method table" What if I did dispatch through defns?
death age. could make overwriting defns sound? That'd be crazy.
a shadowing event in the implicit prefix
exists f1, exists f1

generalize is merely actualizing the implicit foralls as actual foralls.

Other abstract domains that might be nice for variable sets? Intervals, cofinite, top, depth=0 is top.

Graham seemed to immeidately want to use the term "Free Variable" https://en.wikipedia.org/wiki/Free_variables_and_bound_variables . To me free variable is an open lambda body which is rigid, not instantiable. FreshConst / FreshVar distinction makes sense to me. 

In [3]:
max([], default=0)

0

In [6]:
frozenset.intersection(frozenset([1,2]), frozenset([2,3]), frozenset([2,3]))

frozenset({2})

It isn't seaprable
The set of free vars is   `explicit_vars U [age, ...]`



In [ ]:
def merge(x,y):
    agex, fvx = x
    agey, fvy = y
    if agex > agey:
        extra = [v for v in fvy if v.age > agex]
    elif agex < agey:
        extra = [v for v in fvx if v.age > agey]
    max(agex,agey), (fvx & fvy) | extra
    

In [ ]:
def merge(ages):
    newage = 0
    fresh_vars = set()
    for age, fvs in ages:
        newage = max(age, newage)


In [ ]:



type QuantDepth = int
EMPTYSET : frozenset[smt.ExprRef] = frozenset()
class Proof():
    genvars : tuple[QuantDepth, frozenset[smt.ExprRef]] # quantified depth + exceptions. forall excepts, forall xfuture + ...
    # age
    # fresh_vars : frozenset[smt.ExprRef] = EMPTYSET

def merge(x, y):
    qdx, exceptsx = x.genvars
    qdy, exceptsy = y.genvars
    return max(qdx, qdy), exceptsx | exceptsy


def prove(thm, bys):
    qdepth = functools.reduce(merge, (by.genvars[0] for by in bys), (currentdepth, frozenset()))
    Prove(tuple[qdepth, ])

def axiom(thm):
    return Proof((currentdepth, frozenset()))

def generalize(pf, vs):
    qdepth, excepts = pf.genvars
    assert all(v.era >= pf.era or v in pf.genvars[1] for v in vs) # only generalize allowable variables
    return axiom(smt.ForAll(vs, pf.thm), pf.era, pf.fixes) # we do not necessarily lose power here.

def weaken(pf, qdepth=None, fixes=None):
    if qdepth is None:
        qdepth = currentdepth
    else:
        assert qdepth >= pf.genvars[0]
    if fixes is None:
        fixes = frozenset()
    else:
        assert isinstance(fixes, frozenset)
        assert all(fix.qdepth >=  pf.genvars[0] or fix in pf.genvars[1] for fix in fixes)
    return Proof((qdepth, frozenset(fixes)))


def skolem(pf, fixes):
    # skolem loses the ability to generalize. Only things _explicitly_ excepted are generalizable
    # currentdepth is the worst you can ever do.
    FreshFunction
    axiom(      , (currentdepth, frozenset(fixes) | pf.genvars[1]))

def FreshVar(name, sort):
    currentdepth += 1
    FreshEvidence(currentdepth, v)
    return v

In [ ]:

class AllVars(): ...
_ALLVARS = AllVars()

type GenVars = AllVars | frozenset[smt.ExprRef]

class Proof():
    genvars : GenVars # fixes?

def min_genvars(x : GenVars, y : GenVars) -> GenVars:
    if x is _ALLVARS:
        return y
    if y is _ALLVARS:
        return x
    assert isinstance(x, set) and isinstance(y, set)
    return x & y


def prove(fm : smt.BoolRef, by : list[Proof] = [], fixes=_ALLVARS) -> Proof:
    genvars = functools.reduce(min_genvars, (pf.genvars for pf in by), fixes)


def axiom(fm : smt.BoolRef, fixes : GenVars = frozenset()) -> Proof:
    assert isinstance(fm, smt.BoolRef)
    assert genvars is _ALLVARS or (isinstance(genvars, frozenset) and all(is_fresh_var(v) for v in genvars)) 
    return Proof( genvars, smt.Theorem(fm),)

def generalize(vs, pf):
    assert pf.fixes is _ALLVARS or all(v in pf.fixes for v in vs)
    axiom(smt.ForAll(list(vs), pf.thm.body), fixes=pf.fixes)

def weaken(pf, vs):
    assert isinstance(pf, Proof)
    assert pf.fixes is _ALLVARS or all(v in vs for v in pf.fixes)
    return Proof(vs, pf.thm)

def skolem(pf, fixes=frozenset()):
    assert isinstance(pf, Proof)
    assert isinstance(fixes, frozenset) and all(is_fresh_var(v) for v in fixes)
    assert pf.fixes is _ALLVARS or all(v in fixes for v in pf.fixes)
    if len(fixes) == 0:
        skolems =  [
        smt.FreshConst(q.var_sort(i), prefix=q.var_name(i).split("!")[0])
        for i in range(q.num_vars())
        ]
    else:
        genvs = list(fixes)
        sorts = [v.sort() for v in genvs]
        skolems = [smt.FreshFunction(*sorts, q.var_sort(i))(*genvs) for i in range(q.num_vars())]
    








In [ ]:
import logging

logger = logging.getLogger("knuckledragger")
"""
Proof_new = Proof.__new__

def sin_check(cls, thm, reason, admit=False, i_am_a_sinner=False):
    if admit and not config.admit_enabled:
        raise ValueError(
            thm, "was called with admit=True but config.admit_enabled=False"
        )
    if not i_am_a_sinner:
        raise ValueError("Proof is private. Use `kd.prove` or `kd.axiom`")
    return Proof_new(cls, thm, list(reason), admit)


Proof.__new__ = sin_check
"""
    # TODO: This is getting too hairy for the kernel? Reassess. Maybe just a lambda flag? Autolift?
    if lift_lambda and isinstance(body, smt.QuantifierRef) and body.is_lambda():
        # It is worth it to avoid having lambdas in definition.
        vs = fresh_const(body)
        # print(vs, f(*args)[tuple(vs)])
        # print(smt.substitute_vars(body.body(), *vs))
        def_ax = axiom(
            smt.ForAll(
                args + vs,
                smt.Eq(
                    f(*args)[tuple(vs)], smt.substitute_vars(body.body(), *reversed(vs))
                ),
            ),
            by="definition",
        )
    # assert f not in __sig or __sig[f].eq(   def_ax.thm)  # Check for redefinitions. This is kind of painful. Hmm.
    # Soft warning is more pleasant.

def test_lambda_def():
    # test that the lambda has been removed by the definition mechanism
    x, y = smt.Ints("x y")
    z, w = smt.Bools("z w")
    test = kd.define("test", [x], smt.Lambda([x], x), lift_lambda=True)
    assert test.defn.thm.body().eq(smt.ForAll([x, y], test(x)[y] == y).body())


def forget(ts: Iterable[smt.ExprRef], pf: Proof) -> Proof:
    """
    "Forget" a term using existentials. This is existential introduction.
    This could be derived from forget2
    """
    # Hmm. I seem to have rarely been using this
    assert is_proof(pf)
    vs = [smt.FreshConst(t.sort()) for t in ts]
    return axiom(smt.Exists(vs, smt.substitute(pf.thm, *zip(ts, vs))), ["forget", pf])

# modules

https://types.pl/@maxsnew/115305568271533541
a module of a sginature os a model of the signature
STLC[Sigma] parametrized lambda calculus
functors are functors

https://mathstodon.xyz/@olynch/115299725196735176

https://dl.acm.org/doi/pdf/10.1145/800055.802036

https://dl.acm.org/doi/pdf/10.1145/512644.512670  Using Dependent Types to Express Modu|ar Struetr~re

https://link.springer.com/chapter/10.1007/3-540-55719-9_102 A proposed categorical semantics for Pure ML

Currently knuckeldragger is C-like in that everything can clash. discipline can 
resgiter a name as _taken_ at the least.
kd.DeclareFun()

what if defns could be hierarchical? Theorems only apply in the right module? Seems kind of easy to puncture.
Maybe this could also be used to organize simps and auto.

I had notes on modules for egglog


First class modules. If I could internalize a notion of genericity or sorts. The sexp model.

https://pvs.csl.sri.com/doc/interpretations.pdf Interpretations in pvs

"parametric modules" Is that that mnuch different


Using z context as modules. `translate` is kind of module compositions?
Yeah, I dunno.
I don't hate knowing you're in a fresh context.



In [ ]:
class Context(smt.Context):
    def __init__(self, sig=None):
        super().__init__()
        self.sig = []
        self.freshvars = []
    @classmethod
    def combine(self, other): ...

    def Function(self, name, *args):
        return smt.Function(name, *args, ctx=self)
    def Solver(self):
        return smt.Solver(ctx=self)

ctx = Context()
ctx.Solver()





#ctx.Function("foo", smt.IntSort())

[]

Help on function Function in module kdrag.smt:

Function(*args, **kwargs)



A module of funcref and constants + laws can be internalized as a Strcut + wf

wf is kind of "in". Does it need a dual? 
x.asserts = |= x.wf()
x.asserts ~ |= x.assumes

f(x,y).assumes = smt.And(x.assumes, y.assumes)
f(x,y).asserts = |= smt.And(x.assumes, y.assumes) => someprop

FuncDeclRef.__call__ = lambda self, *args: if subtype



p.wf_pf

Keep the wf and assumes separate from the extra ForAll args.

Implies(smt.And(wf, asserts), Impliesypss, conc)

p.wf_pf property is auto modus ponens







# typeclass


Operations come in sortdispatch,
But they aren't held abstract then


FreshVar is maybe analog of isabelle fixes.
So make signature freshvars instead of record.
But freshvars in definitions are tricky.
Implicit parameters to functions... skolemization


In [ ]:
from kdrag.all import *
A = smt.DeclareTypeVar("A")
join = smt.Function("join", A, A, A)
is_inf = smt.Function("is_inf", A, A, A, smt.BoolSort())

join(x,x).decl().domain(0)


Z3Exception: b'domain sort Int and parameter A do not match'

In [4]:
import z3
z3.get_version_string() # 4.15.4
A = z3.DeclareTypeVar("A")
x = z3.Int("x")
y = z3.Const("y", A)
def array_cast(self, x):
    return x
z3.ArraySortRef.cast = array_cast
z3.QuantifierRef.cast = array_cast
z3.Lambda([y], y)[x]
#a = z3.Array("a", A, A)
#a[x]

Z3Exception: b'domain sort Int and parameter A do not match'

In [ ]:
def Lattice(T):
    join = kd.FreshVar("join", smt.ArraySort(T, T, T))


@GenericDispatch
def leq(T):
    return kd.FreshVar("leq", smt.ArraySort(T, T, smt.BoolSort()))

@GenericDispatch
def is_inf(x,y,z):
    return kd.define("is_inf", [x,y,z],
    smt.And(
        z <= x, z <= y,
        smt.ForAll([w], (w <= x) & (w <= y), w <= z)
    ))


@GenericDefine
def is_inf(x,y,z):
    # take parameters that are expressions. Exact MultipleDispatch on all three.
    w = smt.Const("w", x.sort())
    return smt.And(
        z <= x, z <= y,
        smt.ForAll([w], (w <= x) & (w <= y), w <= z)
    )


@GenericDispatch
def is_inf(T):
    x,y,z,inf = smt.Consts("x y z inf", T)
    return kd.define("is_inf", [x,y,inf],
    smt.And(
        inf <= x, inf <= y,
        smt.ForAll([z], z <= x, z <= y, z <= inf)
    ))

def Lattice(T):
    # just a simple combinator
    returns [
        smt.Exists([inf], is_inf(x,y,inf)),
        smt.Exists([sup], is_sup(x,y,sup))
    ]


x,y = kd.FreshVars("x y", smt.IntSort())
join = kd.kernel.obtain(smt.Exists([inf], is_inf(x,y,inf)))
THE = kd.notation.choice


@functools.cache
def Lattice(T):
    # or wrap in a definition
    return kd.define("lattice", [], smt.And(
        smt.ForAll([x,y], smt.Exists([inf], is_inf(x,y,inf))),
        smt.ForAll([x,y], smt.Exists([sup], is_sup(x,y,sup)))
    ))




In [ ]:
def prop_factory(name, ands):
    prop = kd.define(name, [], smt.And(ands))
    selectors = [kd.prove(smt.Implies(prop, a), unfold=1) for a in ands]
    return prop, selectors
def define_prop(name, body):
    if smt.is_and(body):
        ands = [body.arg(i) for i in range(body.num_args())]
    else:
        ands = [body]
    return prop_factory(name, ands)

def define_subprop(name, S, body):
    kd.define(name, [S], smt.Implies(S, body))

def 

def Closed(S : smt.FuncRef, ops):
    smt.And(
        *(smt.ForAll([x,y], S(x) & S(y) >> S(op(x,y)) for op in ops),
          smt.ForAll([x], S(e))
        )
    )

@functools.cache
def monoid(T : smt.ExprRef):
    return prop_factory(f"monoid_{T}", [
        smt.ForAll([x,y,z], (x * y) * z == x * (y * z)),
        smt.ForAll([x], e * x == x),
        smt.ForAll([x], x * e == x)
    ])

In [ ]:
import kdrag as kd
import kdrag.smt as smt
import functools

@functools.cache
def semigroup(T: smt.SortRef):
    x, y, z = smt.Consts("x y z", T)
    return kd.define(
        "semigroup",
        [],
        smt.ForAll([x, y, z], (x * y) * z == x * (y * z)),
    )
def add_semigroup(T: smt.SortRef):
    x, y, z = smt.Consts("x y z", T)
    return kd.define("add_semigroup",[],
        smt.ForAll([x, y, z], (x + y) + z == x + (y + z)))

def comm_semigroup(T: smt.SortRef):
    x, y = smt.Consts("x y", T)
    return kd.define("comm_semigroup", [],
        smt.And(
            semigroup(T),
            smt.ForAll([x, y], x * y == y * x),
        )
    )

def field(prop : smt.BoolRef, n : int):
    pf = kd.unfold(prop.decl())
    return kd.kernel.prove(smt.Implies(prop, pf.thm.children()[n]), by=[pf])



class CommSemigroup():
    def __init__(self, T):
        self.T = T
        self.prop = comm_semigroup(T)

    @functools.cached_property
    def comm(self):
        x, y = smt.Consts("x y", self.T)
        return kd.prove(kd.QForAll([x,y], self.prop, x * y == y * x), unfold=1)
    @functools.cached_property
    def assoc(self):
        x, y,z = smt.Consts("x y z", self.T)
        return kd.prove(kd.QForAll([x,y,z], self.prop, (x * y) * z == x * (y * z)), unfold=1)

CommSemigroup(smt.IntSort()).assoc




|= ForAll([x, y, z], Implies(comm_semigroup, x*y*z == x*y*z))

In [ ]:
@functools.cache
def monoid(T: smt.SortRef):
    """
    >>> monoid = monoid(smt.IntSort())
    >>> e = smt.Const("e", smt.IntSort())
    >>> kd.prove(smt.Implies(monoid, e * e == e), unfold=1)
    """
    e = smt.Const("e", T)
    x, y, z = smt.Consts("x y z", T)
    return kd.define(
        "monoid",
        [],
        smt.And(
            semigroup(T),
            smt.ForAll([x], e * x == x),
            smt.ForAll([x], x * e == x),
        ),
    )

monoid = monoid(smt.IntSort())
e = smt.Const("e", smt.IntSort())
kd.prove(smt.Implies(monoid, e * e == e), unfold=1)
kd.prove(smt.Implies(monoid, e == 1), unfold=1)

In [ ]:
from kdrag.all import *
import functools
@functools.cache
def Monoid(T : smt.SortRef):
    def wf(m):
        mul = m.mul
        e = m.e
        return smt.And(
            smt.ForAll([x,y,z], mul(mul(x,y), z) == mul(x, mul(y,z))),
            smt.ForAll([x], mul(x, e) == x),
            smt.ForAll([x], mul(e, x) == x),
        )
    return kd.Struct(f"Monoid_{T}", ("mul", smt.ArraySort(T,T,T)), ("e", T), pred=wf)




 deriving for newtype

 deriving=arith+[kd.notation.call]

for sd in deriving:
    # derive as a map
    sd.register(dt, lambda *args: dt(sd(*[arg.val for arg in args])))
barely worth doing

In [14]:
import functools
from kdrag.all import *
@functools.cache
def Filter(T : smt.SortRef):
    dt = kd.Struct("Filter", ("sets", smt.SetSort(smt.SetSort(T))))
    A,B = smt.Consts("A B", smt.SetSort(T))
    F = smt.Const("F", dt)
    kd.notation.call.register(dt, lambda F, A: dt.sets(F)[A])
    kd.notation.wf.define([F], smt.And(
            F(smt.FullSet(T)),
            smt.ForAll([A,B], F(A), A <= B, F(B)),
            smt.ForAll([A,B], F(A), F(B), F(A & B)),
        ))
    return dt

FReal = Filter(smt.RealSort())
RSet = smt.SetSort(smt.RealSort())
F = smt.Const("F", FReal)
A,B,C = smt.Consts("A B C", RSet)
@kd.Theorem(kd.QForAll([F], F(smt.FullSet(smt.RealSort()))))
def filter_full(pf):
    pf.unfold()
    pf.auto()
@kd.Theorem(kd.QForAll([F], smt.ForAll([A,B], F(A), A <= B, F(B))))
def filter_mono(pf):
    pf.unfold()
    pf.auto()
@kd.Theorem(kd.QForAll([F], smt.ForAll([A,B], F(A), F(B), F(A & B))))
def filter_inter(pf):
    pf.unfold()
    pf.auto()

class FILTER():
    def __init__(self, T : smt.SortRef):
        self.T = T
        self.S = Filter(T)
        F = smt.Const("F", self.S)
        self.filter_full = kd.prove(kd.QForAll([F], F(smt.FullSet(T))), unfold=1)
        self.filter_mono = kd.prove(kd.QForAll([F], smt.ForAll([A,B], F(A), A <= B, F(B))), unfold=1)
        self.filter_inter = kd.prove(kd.QForAll([F], smt.ForAll([A,B], F(A), F(B), F(A & B))), unfold=1)


In [26]:
Monoid(smt.IntSort())
# a definition that is monoid polymorphic takes in a monoid parameter.


Monoid_Int

Or add a cache_default=False parameter
```
def GenericDispatch(wrapped):


cache_define
but define is kind of cached.



In [ ]:
class GenericDispatch(kd.notation.SortDispatch):
    def __init__(self, def_f, **kwargs):
        self.def_f = def_f
        super().__init__(def_f.__name__, **kwargs)
    def __getitem__(self, T):
        res = self.methods.get(T)
        if res is None:
            res = self.def_f(T)
            self.register(T, res)
        return res
    def __call__(self, *args): 
        T = args[0].sort()
        res = self[T] #self.methods.get(T)
        return res(*args)

@GenericDispatch
def monoid(T : smt.SortRef):
    e = smt.Const("e", T)
    mul = smt.Array("mul", T, T, T)
    x,y,z = smt.Consts("x y z", T)
    return kd.define("monoid", [e,mul], smt.And(
        smt.ForAll([x], mul(x,e) == x)
    ))

e = smt.IntVal(1)
x,y = smt.Ints("x y")
mul = smt.Lambda([x,y], x * y)
kd.rewrite.unfold(monoid(e, mul))
        

And(ForAll(x, Select(Lambda([x, y], x*y), x, 1) == x))

Monoid1 = kd.define("monoid1", [], smt.Exists([e,mul], laws)
monoid2 = kd.define("monoid2, [e, mul], laws)
monoid3 = kd.define("monoid3", [], smt.Lambda([e,mul], laws))



As an idiom

In [ ]:
@functools.cache
def Monoid(T):
    e = smt.Const("e", T)
    mul = smt.Array("mul", T, T)
    return monoid.define([e,mul], smt.And(laws))
monoid =  SortDispatch("monoid", default = lambda e,f: Monoid(e.sort())(e,f))

Doing a define vs just unfolding the define.

In [ ]:
@functools.cache
def Monoid(T):
    e = smt.Const("e", T)
    mul = smt.Array("mul", T, T)
    return smt.And(laws)

This older idea. This was Proof keyed on stuff.
sort dispatch requires an argument.

In [ ]:
class GenericProof:
    """
    GenericProof is a dictionary of proofs indexed on meta (python) information.

    Because z3 and hence knuckledragger does not have strong generic support inside its logic,
    we can borrow a bit of python to parametrize theorems over other data or over sorts.

    This has some of the flavor of single dispatch.

    It is similar to an axiom schema is some respects (in usage it takes in python data and outputs a `kd.Proof`) except that one must register
    the theorems as proven by other means.

    It is a way to refer to a collection of similar proofs akin to single entry typeclasses or modules in other theorem proving systems.

    >>> x = lambda T: smt.Const("x", T)
    >>> obvious = GenericProof(lambda T: smt.ForAll([x(T)], x(T) == x(T)) )
    >>> obvious.lemma(smt.IntSort(), by=[])
    >>> R = smt.RealSort()
    >>> obvious.register(R, kd.prove(smt.ForAll([x(R)], x(R) == x(R))))
    """

    def __init__(self, f):
        self.wrapped = f
        self.data = {}

    def __call__(self, *args) -> kd.Proof:
        return self.data[args]

    def __getitem__(self, *args) -> kd.Proof:
        return self.data[args]

    def __contains__(self, *args) -> bool:
        return args in self.data

    def get(self, *args) -> Optional[kd.Proof]:
        return self.data.get(args)

    def lemma(self, *args, **kwargs):
        return self.register(*args, kd.kernel.prove(self.wrapped(*args), **kwargs))

    def register(self, *args):
        args, pf = args[:-1], args[-1]
        if not kd.kernel.is_proof(pf):
            raise ValueError("Not a proof", pf)
        formula = self.wrapped(*args)
        if not kd.utils.alpha_eq(formula, pf.thm):
            raise ValueError("Proof does not match", formula, pf)
        self.data[args] = pf

Inheriting from superclasses


def Monoid(T):
    return smt.And(SemiGroup(T), other_lwas)







# llvmlite compile
https://github.com/sdiehl/numpile/blob/master/numpile.py



# zig


# sexp

Another idea - 
delaboration via sexp

Also strangely enough, maybe syntactic manipulation might be faster via serialize -> sexp -> deserialize.
Building z3py stuff is just so slow.

Building rewriters over sexp is kind of interesting anyway

The transfer principle?

https://types.pl/@sandmouth/114904803899189629 What I did here is a little different.
Hmm.


s-expressions have set of scope and line number info



In [12]:
class Syntax(list):
    start: int
    end: int
    scope_set = 

s = Syntax([1,2,3])



AttributeError: 'Syntax' object has no attribute 'start'

In [ ]:
Atom = Sexp1.constructor(0)
List = Sexp1.constructor(1)
_ax = smt.Unit(Sexp1.Atom(smt.StringVal("x")))
Unitdecl = _ax.decl()
Concatdecl = smt.Concat(_ax, _ax).decl()
Empty = smt.Empty(smt.SeqSort(Sexp1))


def pprint(e: smt.ExprRef) -> str:
    """
    Pretty print an s-expression.

    >>> pprint(parse("(+ a (b c))"))

    """
    assert e.sort() == Sexp1
    if e.decl() == Atom:
        s = e.arg(0)
        assert smt.is_string_value(s)
        return s.as_string()
    elif e.decl() == List:
        seq = e.arg(0)
        decl = seq.decl()
        if e.eq(Empty):
            return "()"
        elif decl == Unitdecl:
            return "(" + pprint(seq.arg(0)) + ")"
        elif decl == Concatdecl:
            return "(" + " ".join(pprint(c.arg(0)) for c in seq.children()) + ")"
        else:
            raise TypeError(f"Unexpected sequence {seq}")
    else:
        raise TypeError(f"Unexpected expression {e}, {e.decl().name()}")

# nominal
How to do nominal knuckeldragger? Semantic model?

Could help me understand slotted

https://www.cl.cam.ac.uk/~amp12/papers/nomlfo/nomlfo-draft.pdf

https://www.cl.cam.ac.uk/~amp12/talks/tacs01.pdf talk

No, maybe Tuple(Prim, AtomSet)


In [ ]:
# oof this is unsopund in the presence on FreshVars. Huh. Subtle or obvious?
def FreshAtom(xs, prefix=None):
    """
    Axiom schema for there are always new constants.
    >>> FreshAtom([a, b], prefix="fred")
    (fred!..., |= Distinct(fred!..., a, b))
    """
    # TODO: might also need a Seq form of
    z = smt.FreshConst(Atom, prefix=prefix)
    return z, kd.axiom(smt.Distinct(z, *xs))

# But this is the same thing. Uh oh.
def DistinctN(n : int):
    xs = [smt.Const(f"x{i}", Atom) for i in range(n)]
    z = smt.Const("z", Atom)
    return kd.axiom(smt.ForAll([xs], smt.Exists([z], Distinct(z, *xs))))

# Is FreshVar unsound?
# I can prove something exists
# forall exists does not imply exists forall








|= False

In [ ]:
Atom = smt.DeclareSort("Atom")


x,y,z = smt.Consts("x y z", Atom)
kd.define("swap", [x,y,z], 
    smt.If(x == z, y,
    smt.If(y == z, x,
    z))
)

swap = kd.notation.SortDispatch(name="swap")
free = kd.notation.SortDispatch(name="free")

swap.register()

def NomSort(s : smt.SortRef) -> smt.SortRef:
    dt = kd.Inductive("Nom_" + s.name(), [s])
    dt.declare("Atom", ("atom", Atom)) # Set(Atom) ? Var?
    dt.declare("Value", ("prim", s))
    dt = dt.create()
    v = smt.Const("v", dt)
    swap.define([s,x,y], 
        smt.If(dt.is_Atom(v), dt.Atom(swap(x,y,dt.prim(v))),
        dt.Value(v))
    )
    # v # t
    distinct.define([s,x],
        smt.If(dt.is_Atom(v), v != dt.atom(x),
        smt.BoolVal(False)
    ))

    return dt.create(), swap, free



#  ??????.  phi(x,y,z) ?    (Bool, Set(Atom)) ? Perm -> Bool ?  
Bool = NormSort(smt.BoolSort())

f = smt.ArrayConst("f", Atom, Atom)
is_iso = kd.define("is_iso", [f], 
    smt.ForAll([x],
        smt.Exists([y],
            f[x] == y
        )
    )
)
compact = kd.define("compact", [f],
    smt.Exists([seq],
        f[a] == smt.In(seq, a)
    )
    smt.ForAll([x,y,z],
        smt.Implies(f[x] == f[y], x == y)
    )
)

perm_inv = kd.define("perm_inv", [a],
    smt.ForAll([x,y],
        smt.Implies(free(a,x) == free(a,y), x == y)
    )
)




In [ ]:
NomVar = smt.DeclareSort("NomVar")

def Nominal(S : smt.SortRef):
    dt = kd.Inductive("Nom_" + S.name())
    dt.declare("Nom", ("var", NomVar))
    dt.declare("Const", ("val", S))
    return dt.create()

# not good enoguh for recursive trees.

def NomDT(name):
    dt = kd.Inductive("name")
    dt.declare("Var", ("var", NomVar))
    return dt

# but also it has a different 

# Conv

Conv could be nice way to use zippers + amgnual congruence rules.
Which in turn let's us maybe get a handle on generalized congruence.

Lenses?


https://pavpanchekha.com/blog/zippers/multi-zippers.html
https://stackoverflow.com/questions/3436559/zipper-like-data-structure-with-more-than-one-cursor
https://okmij.org/ftp/continuations/zipper.html#zipper2 zippers with several holes



# Knuckle Kernels
a monorepo of knuckeldragger kernels



# Delab

lisp2tex?


pprint?






In [ ]:

pprint_binop = {
    "and": r"\land",
    "or": r"\lor",
    "not": r"\lnot",
    "=>": r"\implies",
    "=": "=",
    "distinct": r"\neq",
    "+": "+",
    "-": "-",
    "*": r"\cdot",
    "/": "/",
    "<": "<",
    "<=": r"\leq",
    ">": ">",
    ">=": r"\geq",
}
pprint_unop = {
    "not": r"\lnot",
    "-": "-",
}



#def pprint(sexp):
def latex(sexp):
    match sexp:
        case [f, *args]:



# macro smt


# parametrized solvers

grind can take in theories


In [ ]:
from typing import Protocol
class Theory(Protocol):
    decls : list[smt.FuncDeclRef]
    def init(self) -> list[smt.BoolRef]: ...
    def preprocess(self, formula : smt.BoolRef):
        # callback from Solver.add

class Ring(Protocol):
    ...

class RingTheory(Theory):
    ring : Ring

    def preprocess(self, formula): ...
        # do ring normalization
    def simplify(self, e : smt.ExprRef): ...

class Ring(Protocol, Theory):



class Solver():
    solver : smt.Solver
    theories : list[Theory]
    def add_theory(self, t : Theory):
        self.theories.add(t)
        for e in t.init():
            self.solver.add(e)
    def add(self, formula):
        self.solver.add(t)
        for theory in theories:
            for t in theory.preprocess(formula):
                self.solver.add(t)
    def check(self):
        res = self.solver.check()
        if res == smt.unsat:
            return smt.unsat
        elif res == smt.sat:
            m = self.solver.get_model()
            for theory in self.theories:
                theory.check(m)




In [14]:
def bysect(thm, by, **kwargs):
    if isinstance(by, list):
        by = list(enumerate(by))
    elif isinstance(by, dict):
        by = list(by.items())
    n = 2
    while len(by) >= 2:
        subset_size = len(by) // n
        for i in range(0, len(by), subset_size):
            part = by[i:i+subset_size]
            rest = by[:i] + by[i+subset_size:]
            try:
                kd.prove(thm, by=[b for _,b in rest], **kwargs)
                by = rest
                n = max(n-1, 2)
                found = True
                break
            except Exception as e:
                pass
        else:
            if n == len(by): break
            n = min(len(by), n*2)
    return by

x,y,z = smt.Ints("x y z")
by = [
kd.prove(x + 1 > x),
kd.axiom(x == y),
#kd.axiom(z == x),
kd.prove(y + 1 > y),
kd.axiom(y == z),
]

bysect(x == z, by=by)

by = {
"x1" : kd.prove(x + 1 > x),
"xy" : kd.axiom(x == y),
#kd.axiom(z == x),
"y1" : kd.prove(y + 1 > y),
"yz" : kd.axiom(y == z),
}
bysect(x == z, by=by)


[('xy', |= x == y), ('yz', |= y == z)]

# boolseq intseq

Was I onto the right idea with genericproof / genericdef?

GTheorem() # GenericTheorem
def foobiz(*params):
    l = kd.Lemma(yada)
    return l.auto()



    

In [ ]:
class Countable():
    def __init__(self, A : smt.SortRef):
        x,y = smt.Consts("x y", sort)
        f = smt.Array("f", smt.IntSort(), s)
        n = smt.Int("n")
        s = smt.ArraySort(sort, smt.BoolSort())
        self.countable = kd.define()

        kd.Lemma(smt.Implies(countable[A], coumtable[B], countable[A | B]))
        self.countable_union =
        countable_inter = 

        has_card(A, N) = smt.Exissts([f,g], smt.ForAll([x], smt.And(f[g[x]] == x, g[x] < N, 0 < g[x])), smt.ForAll([n], 0 <= n, n < M, f[g[n]] == n, )


    deg __init__

turn Set into a module style?



In [8]:
from kdrag.all import *
import functools
from kdrag.theories.set import *
#@functools.cache
def _countable(setsort : smt.SortRef):
    sort = setsort.domain()
    f = smt.Array("f", smt.IntSort(), sort)
    n = smt.Int("n")
    s = smt.Array("s", sort, smt.BoolSort())
    y = smt.Const("y", sort)
    y.assumes = s[y]
    print("hello")
    return countable.define([s], smt.Exists([f], kd.QForAll([y], smt.Exists([n], f[n] == y))))
countable = kd.notation.SortDispatch(name="countable", default=lambda s: _countable(s.sort())(s))

s = smt.Const("s", RSet)
countable(s)
countable(s)
countable(s)

hello


countable(s)

In [4]:
from kdrag.all import *
#import kdrag.theories.logic.axioms as ax
BSeq = smt.ArraySort(smt.IntSort(), smt.BoolSort())
ISeq = smt.ArraySort(smt.IntSort(), smt.IntSort())

import functools

#@functools.cache
def _countable(sort : smt.SortRef):
    x,y = smt.Consts("x y", sort)
    f = smt.Array("f", smt.IntSort(), s)
    n = smt.Int("n")
    s = smt.ArraySort(sort, smt.BoolSort())
    return kd.define("countable", [s], smt.Exists([f], kd.QForAll([y], s[y], smt.Exists([n], f[n] == y))))
countable = kd.notation.SortDispatch(name="countable", default=lambda s: _countable(s.sort())(s))

#def countable(s : ExprRef):
#    return _countable(s.domain())(s)


enumerates = kd.notation.SortDispatch(name="enumerates")
#enumerates.define([])


@kd.Theorem(countable(smt.IntSort()))
def countable_int(l):
    n = smt.Int("n")
    l.unfold()
    l.exists(smt.Lambda([n], n))
    l.auto()

@kd.Theorem(countable(smt.BoolSort()))
def countable_bool(l):
    n = smt.Int("n")
    l.unfold()
    l.exists(smt.Lambda([n], smt.If(n % 2 == 0, smt.BoolVal(True), smt.BoolVal(False))))
    l.auto()

@kd.Theorem(smt.Not(countable(smt.ArraySort(smt.IntSort(), smt.BoolSort()))))
def not_countable_boolseq(l):
    l.unfold()
    l.intros()
    f = l.obtain(0)
    n = smt.Int("n")
    l.instan(0, smt.Lambda([n], smt.Not(f[n][n])))
    n1 = l.obtain(1)
    l.ext(at=1)
    l.auto()

@kd.Theorem(smt.Not(countable(smt.ArraySort(smt.IntSort(), smt.IntSort()))))
def not_countable_intseq(l):
    l.unfold()
    l.intros()
    f = l.obtain(0)
    n = smt.Int("n")
    l.instan(0, smt.Lambda([n], f[n][n] + 1))
    l.clear(0)
    n1 = l.obtain(0)
    l.ext(at=0)
    l.auto()

not_countable_intseq

|= Not(countable)

In [ ]:
            if smt.is_array_sort(lhs):
                self.pop_goal()
                ext_ind = smt.Ext(lhs, rhs)
                x = smt.FreshConst(ext_ind.sort())
                newgoal = smt.Eq(lhs[x], rhs[x])
                self.add_lemma(
                    kd.kernel.prove(
                        smt.Implies(x == ext_ind, smt.Eq(lhs, rhs) == newgoal)
                    )
                )
                self.goals.append(
                    goalctx._replace(ctx=goalctx.ctx + [x == ext_ind], goal=newgoal)
                )
                return x

In [ ]:
@functools.cache
def finset(sort : smt.SortRef):
    s = smt.Const("s", smt.SetSort(sort))
    n, N = smt.Int("n N")
    f = smt.Array("f", smt.IntSort(), sort)
    return kd.define("finset", [s], smt.Exists([f,N], smt.QForAll([y]), s[y], kd.QExists([n], 0 <= n, n < N, f[n] == y))

finset()


# delta satisfiability

https://scungao.github.io/papers/delta_complete_decision_procedures.pdf

nonstyandard reals
lifting

nate says maxmimize delta is nice.

Exact reals





In [ ]:

ExactReal = smt.ArraySort(smt.RealSort(), smt.RealSort())

smt.Lambda([eps])


In [ ]:

def unfold_old(e: smt.ExprRef, decls=None, trace=None) -> smt.ExprRef:
    """
    Do a single unfold sweep, unfolding definitions defined by `kd.define`.
    The optional trace parameter will record proof along the way.
    `decls` is an optional list of declarations to unfold. If None, all definitions are unfolded.

    >>> x = smt.Int("x")
    >>> f = kd.define("f", [x], x + 42)
    >>> trace = []
    >>> unfold_old(f(1), trace=trace)
    1 + 42
    >>> trace
    [|= f(1) == 1 + 42]

    >>> unfold_old(smt.Lambda([x], f(x)))
    Lambda(x, x + 42)
    """
    if smt.is_app(e):
        decl = e.decl()
        children = [unfold(c, decls=decls, trace=trace) for c in e.children()]
        defn = kd.kernel.defns.get(decl)
        if defn is not None and (decls is None or decl in decls):
            e1 = smt.substitute(defn.body, *zip(defn.args, children))
            e = e1
            if trace is not None:
                if isinstance(defn.ax.thm, smt.QuantifierRef):
                    trace.append((defn.ax(*children)))
                # else:
                # TODO: It would be better to only emit the forall axiom when necessary.
                # When we have traversed under a binder, things get complicated.
                # instantiating the axiom on the fresh variable will not be useful.
                # so just emitting the needed axiom is a hack for now.
                # trace.append(defn.ax)
            return e1
        else:
            return decl(*children)
    elif isinstance(e, smt.QuantifierRef):
        vs0 = [smt.Const(e.var_name(i), e.var_sort(i)) for i in range(e.num_vars())]
        vs, e1 = kd.utils.open_binder(e)
        if trace is not None:
            subtrace = []
            e2 = unfold(e1, decls=decls, trace=subtrace)
            if not e2.eq(e1):
                trace.extend([pf.forall(vs) for pf in subtrace])
        else:
            e2 = unfold(e1, decls=decls, trace=None)
        if e.is_forall():
            # TODO: When we go under a quantifier, any trace breadcrumb should really be re-quantified in order to be useful.
            # maybe herbrandize?
            e3 = smt.ForAll(vs, e2)
        elif e.is_exists():
            e3 = smt.Exists(vs, e2)
        elif e.is_lambda():
            e3 = smt.Lambda(vs, e2)
        else:
            raise Exception("Unexpected quantifier", e)
        t, pf = kd.kernel.rename_vars(e3, vs0)
        if trace is not None and not t.eq(e):
            trace.append(pf)
        return t
    else:
        return e

# analysis

rewrite should be improved to go under binders.
The rewrite tactic doesn't use rewrite facilities?



In [8]:
from kdrag.all import *
import kdrag.theories.real as real
from kdrag.theories.real.seq import *
@kd.Theorem(smt.Implies(has_lim(a, x) & has_lim(b, y), has_lim(a + b, x + y)))
def has_lim_add(l):
    l.unfold(has_lim)
    l.intros()
    l.split(0)
    eps = l.fix()
    l.intros()
    l.have(smt.Exists([N], kd.QForAll([n], n > N, real.abs(a[n] - x) < eps / 2)))
    l.apply(0)
    l.auto()
    N1 = l.obtain(-1)
    l.have(smt.Exists([N], kd.QForAll([n], n > N, real.abs(b[n] - y) < eps / 2)))
    l.apply(1)
    l.auto()
    N2 = l.obtain(-1)
    l.exists(smt.If(N1 > N2, N1, N2))
    n1 = l.fix()
    l.instan(0, n1)
    l.instan(1, n1)
    l.unfold(add)
    l.auto(by=[real.abs.defn])


@kd.PTheorem(smt.Implies(has_lim(a, x) & has_lim(b, y), has_lim(a - b, x - y)))
def has_lim_sub(l):
    l.unfold(has_lim)
    l.intros()
    l.split(0)
    eps = l.fix()
    l.intros()
    l.have(smt.Exists([N], kd.QForAll([n], n > N, real.abs(a[n] - x) < eps / 2)))
    l.apply(0)
    l.auto()
    N1 = l.obtain(-1)
    l.have(smt.Exists([N], kd.QForAll([n], n > N, real.abs(b[n] - y) < eps / 2)))
    l.apply(1)
    l.auto()
    N2 = l.obtain(-1)
    l.exists(smt.If(N1 > N2, N1, N2))
    n1 = l.fix()
    l.instan(0, n1)
    l.instan(1, n1)
    l.unfold(sub)
    l.auto(by=[real.abs.defn])

Lemma Complete! Change PTheorem to Theorem


In [1]:
@kd.PTheorem(smt.Implies(has_lim(a,x) & has_lim(b, y), has_lim(a + b, x + y)))
def has_lim_add(l):
    l.unfold(has_lim)
    l.intros()
    l.split(0)
    eps = l.fix()
    l.intros()
    l.have(smt.Exists([N], kd.QForAll([n], n > N, real.abs(a[n] - x) < eps / 2)))
    l.apply(0)
    l.auto()
    N1 = l.obtain(-1)
    l.have(smt.Exists([N], kd.QForAll([n], n > N, real.abs(b[n] - y) < eps / 2)))
    l.apply(1)
    l.auto()
    N2 = l.obtain(-1)
    l.exists(smt.If(N1 > N2, N1, N2))
    n1 = l.fix()
    l.instan(0,n1)
    l.instan(1,n1)
    l.unfold(add)
    l.auto(by=[real.abs.defn])





NameError: name 'kd' is not defined

In [ ]:
def lift_lim(lim, N, by=[]):


Once I have more than 3 facts in context, giving them labels would be nice in printing.


In [4]:

from kdrag.all import *
import kdrag.theories.real as real
from kdrag.theories.real.seq import *

@kd.PTheorem(smt.Implies(has_lim(a,x) & has_lim(b, y), has_lim(a - b, x - y)))
def has_lim_sub(l):
    l.unfold(has_lim)
    l.intros()
    l.split(0)
    eps = l.fix()
    l.intros()
    l.have(smt.Exists([N], kd.QForAll([n], n > N, real.abs(a[n] - x) < eps / 2)))
    l.apply(0)
    l.auto()
    N1 = l.obtain(-1)
    l.have(smt.Exists([N], kd.QForAll([n], n > N, real.abs(b[n] - y) < eps / 2)))
    l.apply(1)
    l.auto()
    N2 = l.obtain(-1)
    l.exists(smt.If(N1 > N2, N1, N2))
    n1 = l.fix()
    l.instan(0,n1)
    l.instan(1,n1)
    l.unfold(sub)
    l.auto(by=[real.abs.defn])

Lemma Complete! Change PTheorem to Theorem


# C parsing

In [15]:
from pycparser import parse_file, c_generator
import pycparser

text = """
int myadd(int x, int y) {
    return x + y;
}
"""

parser = pycparser.c_parser.CParser()
ast = parser.parse(text)
ast.show()
generator = c_generator.CGenerator()
generator.visit(ast)

FileAST: 
  FuncDef: 
    Decl: myadd, [], [], [], []
      FuncDecl: 
        ParamList: 
          Decl: x, [], [], [], []
            TypeDecl: x, [], None
              IdentifierType: ['int']
          Decl: y, [], [], [], []
            TypeDecl: y, [], None
              IdentifierType: ['int']
        TypeDecl: myadd, [], None
          IdentifierType: ['int']
    Compound: 
      Return: 
        BinaryOp: +
          ID: x
          ID: y


'int myadd(int x, int y)\n{\n  return x + y;\n}\n\n'

# hyperreal
singledispatch to_expr
default ArraySort to pointwise

intutionsitc loigc prover
triggers

def rw_trigger(kd.Proof):
def horn_trigger(kd.Proof):

mod.__dict__
bysect paramerter


propphecy vars
p





# rewrite
forall cong

going into branches of if, we collect local cond.
Wait but actually a lot of this is hypothetical.
and if we need to 

def hyp_cong(f):
    ks.axiom(t1 == t2 => f(t1) == f(t1))

The analog of herb

def lambda_cong():
    vs, kd.axiom(body1 == body2 -> lambda vs, body = lambda vs, body2)
def subst():
    lhs == rhs -> t1 == t2


tacticals
repeat
branching subgoals. apply rw at each

make subproofs and them meld them?



pmatch returns zipper
zipper stores original




In [3]:
from kdrag.all import *
from kdrag.utils import *
import itertools
def pmatch_zipper(vs, pat, t):
    z = Zipper([], t)
    for t in itertools.chain([t], z):
        subst = pmatch(vs,pat,t)
        if subst is not None:
            return subst, z

x,y,z,a,b,c = smt.Ints("x y z a b c")
pmatch_zipper([a,b], a*b, x + smt.Lambda([y], y*z)[x])

def rewrite_zipper(vs, lhs, rhs, t):
    subst, z = pmatch_zipper(vs, lhs, t)
    if subst is None:
        return None
    else:
        z.t = smt.substitute(rhs, *zip(vs, [subst[v] for v in vs]))
        while z.ctx:
            if z.ctx[-1]:

            z.up()
        return z.rebuild()



({b: z, a: Y!112},
 Zipper(ctx=[DeclHole(f=+, _left=(x,), _right=()), DeclHole(f=Select, _left=(), _right=(x,)), LambdaHole(vs=[Y!112], orig_vs=[y])], t=Y!112*z))

In [ ]:
def pmatch_rec_ctx(
    vs: list[smt.ExprRef], pat: smt.ExprRef, t: smt.ExprRef
) -> Optional[tuple[smt.ExprRef, dict[smt.ExprRef, smt.ExprRef]]]:
    todo = [(ExprCtx(), t)]
    while todo:
        t = todo.pop()
        subst = pmatch(vs, pat, t)
        if subst is not None:
            return t, subst
        elif smt.is_app(t):
            todo.extend(t.children())
        elif (
            isinstance(t, smt.QuantifierRef) and into_binder
        ):  # going into the binder is dicey
            todo.append(t.body())

In [ ]:
def rewrite1_rec(t, vs, lhs, rhs) -> Optional[kd.Proof]:
    """

    >>> x,y = smt.Ints("x y")
    >>> pf = kd.prove(smt.ForAll([x], x + 0 == x))
    >>> rewrite1_rec(x + 0 + 0, [x], x + 0, x)
    """

    def worker(t):
        subst = utils.pmatch(vs, lhs, t)
        if subst is not None:
            return kd.kernel.instan(pf, [subst[v] for v in vs])
        elif isinstance(t, smt.QuantifierRef):
            vs1, body = kd.utils.open_binder_unhygienic(t)
            pf = worker(body)
            if pf is None:
                return None
            else:
                pf = kd.kernel.subst(t, pf)
                if t.is_forall():
                    return kd.kernel.forall_cong(vs1, pf)
                elif t.is_exists():
                    return kd.kernel.exists_cong(vs1, pf)
                elif t.is_lambda():
                    return kd.kernel.lambda_cong(vs1, pf)
                else:
                    raise Exception("Unexpected quantifier", t)
        elif smt.is_app(t):
            for c in t.children():
                pf = worker(c)
                if pf is not None:
                    return pf
            return None
        else:
            raise Exception("Unexpected term in rewrite1", t)

    pf = worker(t)
    if pf is not None:
        return kd.kernel.subst(t, pf)
    else:
        return None

In [ ]:
def rewrite1_rec(t, pf : kd.Proof) -> Optional[kd.Proof]:
    """

    >>> x,y = smt.Ints("x y")
    >>> pf = kd.prove(smt.ForAll([x], x + 0 == x))
    >>> rewrite1_rec(x + 0 + 0, [x], x + 0, x)
    """

    if isinstance(pf, kd)
    vs, body = kd.utils.open_binder_unhygienic(pf.thm)
    assert smt.is_eq(body)
    lhs = body.arg(0)

    def worker(t):
        subst = utils.pmatch(vs, lhs, t)
        if subst is not None:
            return kd.kernel.instan(pf, [subst[v] for v in vs])
        elif isinstance(t, smt.QuantifierRef):
            vs1, body = kd.utils.open_binder_unhygienic(t)
            pf = worker(body)
            if pf is None:
                return None
            else:
                pf = kd.kernel.subst(t, pf)
                if t.is_forall():
                    return kd.kernel.forall_cong(vs1, pf)
                elif t.is_exists():
                    return kd.kernel.exists_cong(vs1, pf)
                elif t.is_lambda():
                    return kd.kernel.lambda_cong(vs1, pf)
                else:
                    raise Exception("Unexpected quantifier", t)
        elif smt.is_app(t):
            for c in t.children():
                pf = worker(c)
                if pf is not None:
                    return pf
            return None
        else:
            raise Exception("Unexpected term in rewrite1", t)

    pf = worker(t)
    if pf is not None:
        return kd.kernel.subst(t, pf)
    else:
        return None

In [ ]:

def rewrite1_rec(t, pf : kd.Proof) -> Optional[kd.Proof]:
    """

    >>> x,y = smt.Ints("x y")
    >>> pf = kd.prove(smt.ForAll([x], x + 0 == x))
    >>> rewrite1_rec(x + 0 + 0, [x], x + 0, x)
    """

    if isinstance(pf, kd)
    vs, body = kd.utils.open_binder_unhygienic(pf.thm)
    assert smt.is_eq(body)
    lhs = body.arg(0)

    def worker(t):
        subst = utils.pmatch(vs, lhs, t)
        if subst is not None:
            return kd.kernel.instan(pf, [subst[v] for v in vs])
        elif isinstance(t, smt.QuantifierRef):
            vs1, body = kd.utils.open_binder_unhygienic(t)
            pf = worker(body)
            if pf is None:
                return None
            else:
                pf = kd.kernel.subst(t, pf)
                if t.is_forall():
                    return kd.kernel.forall_cong(vs1, pf)
                elif t.is_exists():
                    return kd.kernel.exists_cong(vs1, pf)
                elif t.is_lambda():
                    return kd.kernel.lambda_cong(vs1, pf)
                else:
                    raise Exception("Unexpected quantifier", t)
        elif smt.is_app(t):
            for c in t.children():
                pf = worker(c)
                if pf is not None:
                    return pf
            return None
        else:
            raise Exception("Unexpected term in rewrite1", t)

    pf = worker(t)
    if pf is not None:
        return kd.kernel.subst(t, pf)
    else:
        return None


In [ ]:



def cond_rw(): ...

# Like have. Auto discharge via by. otherwise add to new goal to prove precondition
#




# CF Known Bit


Also intervbals

Hmm. What if there are empty intervals. Then the output might be 

So tightest bound theorem might not be true.

In


There are two notions of le
le considered as sets
le considered as a question about the ordering on bitvectors. but this comes in signed and unsigned versions


use `local` to get all names in current scope.



In [ ]:
{name : v for name, v in locals().items() if isinstance(v, kd.Proof) and "elem_" in name}

In [1]:
from kdrag.all import *
from kdrag.contrib.absint.knownbit import *

#kd.simp(~KnownBits.mk(0,-1))

#kd.prove(smt.ForAll([a], ~const(a) == const(~a)), unfold=1)
kd.prove(~top == top, unfold=1)

concr = setof
abstr = ofset

def sound(f, g):
    return smt.ForAll([a], f(abstr(a)) <= concr(g(a)))



|= invert(KnownBits(0, 255)) == KnownBits(0, 255)

In [ ]:
le = kd.notation.le.define([x,y], setof(x) <= setof(y))
ofset = smt.Function("ofset", smt.SetSort(KnownBits), KnownBits)
# ofset(setof(x)) == x
# setof(ofset(s)) == 
# setof = smt.Concat(kd.QExists([x], s[x], x & (smt.BitVecVal(1,W) << i))) == 0,  smt.If(smt.Exists[x,y],  x & (smt.BitVecVal(1,W) << i))) == y & (smt.BitVeclVal(1,W) << i)))

# bvand across entire set
bigbvand = kd.define("bigbvand", [s], Concat(smt.Lambda([i], smt.If(s[i], smt.BitVecVal(1,1), smt.BitVecVal(0,W))), smt.BitVecVal(0,W)))
bigbvor = kd.define("bigbvor", [s], Concat(smt.Lambda([i], smt.If(s[i], smt.BitVecVal(1,W), smt.BitVecVal(0,W))), smt.BitVecVal(0,W)))
# yikes
bigbvadd = 


In [5]:
(~y.ones)
x.unknowns

unknowns(x)

In [ ]:
smt.And([smt.Implies(smt.Extract(i,i,x.unknowns) == 1, smt.Extract(i,i,y.unknowns) == 1), 
        smt.Implies(smt.Extract(i,i,y))    for i in range(W)])

In [6]:
le = kd.notation.le.define([x,y], smt.And((~(x.unknowns)) | y.unknowns == -1,  x.unknowns | ((~y.ones) ^ x.ones) == -1))
# xunk -> yunk
# not(xunk) -> (yones ^ xones == 0)

In [18]:
f = smt.Array("f", KnownBits, KnownBits)
best_invert = kd.QForAll([f], kd.QForAll([x,a], x[a], f(x)[~a]), kd.QForAll([x], smt.IsSubset(setof(~x), setof(f(x)))))
# for all sound operations, our defined one is better
best_invert

ForAll(f,
       Implies(ForAll([x, a],
                      Implies(And(wf(x), setof(x)[a]),
                              setof(f[x])[~a])),
               ForAll(x,
                      Implies(wf(x),
                              subset(setof(invert(x)),
                                     setof(f[x]))))))

In [26]:
l = kd.Lemma(kd.QForAll([f], kd.QForAll([x,a], x[a], f(x)[~a]), kd.QForAll([x], smt.IsSubset(setof(~x), setof(f(x))))))
_f = l.fix()
l.intros()
_x = l.fix()
l.intros()
l

[f!249, x!250];
[ForAll([x, a],
       Implies(And(wf(x), setof(x)[a]), setof(f!249[x])[~a])),
 wf(x!250)]
?|= subset(setof(invert(x!250)), setof(f!249[x!250]))

In [13]:
kd.prove(smt.ForAll([x,y], (x <= y) == (smt.IsSubset(setof(x),setof(y)))), unfold=2, timeout=10000)

LemmaError: (le(x, y) == subset(setof(x), setof(y)), 'Countermodel', [y = KnownBits(0, 0),
 x = KnownBits(4, 0),
 le = [else -> True],
 Ext = [(K(BitVec(8), False), as-array) -> 4, else -> 0],
 setof = [KnownBits(4, 0) ->
          Store(K(BitVec(8), False), 0, True),
          KnownBits(0, 0) ->
          Store(K(BitVec(8), False), 4, True),
          else -> Store(K(BitVec(8), True), 4, False)],
 k!235 = [4 -> True, else -> False]])

In [12]:
ofset = smt.Function("ofset", smt.SetSort(KnownBits), KnownBits)
ofset_setof = kd.axiom(smt.ForAll([x], ofset(setof(x)) == x))


Z3Exception: Sort mismatch

In [10]:
s = smt.Const("s", smt.SetSort(smt.BitVecSort(W)))
unknown = smt.Concat([smt.If(kd.QExists([a,b], s[a], s[b], smt.Extract(i,i,a) != smt.Extract(i,i,b)), smt.BitVecVal(1,1), smt.BitVecVal(0,1)) for i in range(W)])
unknown
ones = smt.Concat([smt.If(kd.QExists([a], s[a], smt.Extract(i,i,a) == smt.BitVecVal(1,1)), smt.BitVecVal(1,1), smt.BitVecVal(0,1)) for i in range(W)])

Concat(Concat(Concat(Concat(Concat(Concat(Concat(If(Exists([a,
                                        b],
                                        And(s[a],
                                        s[b],
                                        Extract(0, 0, a) !=
                                        Extract(0, 0, b))),
                                        1,
                                        0),
                                        If(Exists([a, b],
                                        And(s[a],
                                        s[b],
                                        Extract(1, 1, a) !=
                                        Extract(1, 1, b))),
                                        1,
                                        0)),
                                        If(Exists([a, b],
                                        And(s[a],
                                        s[b],
                                        Extract(2, 2, a) !=
                                        Extract(2, 2, b))),
                                        1,
                                        0)),
                                   If(Exists([a, b],
                                        And(s[a],
                                        s[b],
                                        Extract(3, 3, a) !=
                                        Extract(3, 3, b))),
                                      1,
                                      0)),
                            If(Exists([a, b],
                                      And(s[a],
                                        s[b],
                                        Extract(4, 4, a) !=
                                        Extract(4, 4, b))),
                               1,
                               0)),
                     If(Exists([a, b],
                               And(s[a],
                                   s[b],
                                   Extract(5, 5, a) !=
                                   Extract(5, 5, b))),
                        1,
                        0)),
              If(Exists([a, b],
                        And(s[a],
                            s[b],
                            Extract(6, 6, a) !=
                            Extract(6, 6, b))),
                 1,
                 0)),
       If(Exists([a, b],
                 And(s[a],
                     s[b],
                     Extract(7, 7, a) != Extract(7, 7, b))),
          1,
          0))

In [4]:

l = kd.Lemma(kd.QForAll([x, a], x[a], (~x)[~a]))
_x, _a = l.fixes()
l.unfold()
l.auto()

Nothing to do!

In [ ]:
EInt = kd.Inductive("EInt")
EInt.declare("Const", ("val", smt.IntSort()))
EInt.declare("Inf")
EInt.declare("NegInf")
EInt = EInt.create()

IInterval = kd.Struct("IInterval", ("lo", smt.IntSort()), ("hi", smt.IntSort()), pred=lambda i: smt.Or(i.lo <= i.hi, i.lo == 0, i.lo == 1))
is_empty = kd.define("is_empty", [I], i.hi > i.lo)
is_non_empty = kd.define("is_non_empty", [I], i.hi <= i.lo)
# redundant empty intervals.



In [2]:
from kdrag.all import *
from kdrag.theories.real.interval import *




ISeq = smt.ArraySort(smt.IntSort(), Interval)

a = smt.Const("a", ISeq)
#is_funnel =
#cuminter = kd.define("cuminter", [a], smt.Lambda([n], smt.If(n >= 0, a[n] + cuminter(n - 1)[n], smt.If(n == 0, a[0], ) )))
#is_conv = kd.define("is_conv", [a], smt.ForAll([eps], eps > 0, smt.Exists([N], )))

def Min(xs):
    if len(xs) == 1:
        return xs[0]
    else:
        rest = xs[1:]
        return smt.If(smt.And(xs[0] <= x for x in rest), xs[0], Min(rest))

def Max(xs):
    if len(xs) == 1:
        return xs[0]
    else:
        rest = xs[1:]
        return smt.If(smt.And(xs[0] >= x for x in rest), xs[0], Min(rest))
x,y,z = smt.Ints("x y z")

mul = kd.define("mul", [I,J], Interval.mk(Min([I.hi * J.hi, I.lo * J.lo, I.hi * J.lo, I.lo * J.hi]),
                                 Max([I.hi * J.hi, I.lo * J.lo, I.hi * J.lo, I.lo * J.hi])))

kd.prove(Min([x,y,z]) <= x)
kd.prove(Min([x,y,z]) <= z)



|= If(And(x <= y, x <= z), x, If(And(y <= z), y, z)) <= z

# Category


In [ ]:
Ob = smt.DeclareSort("AOb")
Arr = smt.DeclareSort("AArr")
x,y,z,w = smt.Consts("x y z w", Ob)
f,g,h,k = smt.Consts("f g h k", Arr)
cod = smt.Function("cod", Arr, Ob)
dom = smt.Function("dom", Arr, Ob)
hom = kd.define("hom", [x,y], smt.Lambda([f], smt.And(dom(f) == x, cod(f) == y))) # homset
#smt.Const("hom", smt.ArraySort(Ob, Ob, smt.BoolSort()))
f = smt.Const("f", hom(x,y))
f = smt.Const("f", hom(cod(f), dom(f)))
f = smt.Const("f", smt.Lambda([g], dom(g) == x, cod(g) == y)) # immediate unfold
g = smt.Const("g", hom(y,z))
h = smt.Const("h", hom(z,w))

comp = smt.Function("comp", Arr,Arr,Arr)
kd.notation.matmul.register(Arr, comp)
kd.axiom(smt.ForAll([x,y,z,f,g], hom(x,z)[f @ g]))
id = smt.Function("id", Ob, Arr)
# including x y is ugly and hurtful to quantifier instan probably.
id_left = kd.axiom(smt.ForAll([x,y,f], f @ id(x) == f))
id_right = kd.axiom(smt.ForAll([x,y,f], id(y) @ f == f))
id_hom = kd.axiom(smt.ForAll([x], hom(x,x)[id(x)]))
comp_assoc = kd.axiom(smt.ForAll([w,x,y,z,f,g,h], (f @ g) @ h == f @ (g @ h)))
comp_assoc


C = smt.Array("C", Arr, smt.BoolSort())
C = smt.Const("C", smt.Lambda([S], smt.And(
    smt.ForAll([f,g], C[f], C[g], C[f @ g]),
    smt.ForAll([f], C[f], C[id(dom(f))] & C[id(cod(f))]),
)))
Chom = hom(x,y) & C
f.assumes = hom(x,y) & C
#smt.ForAll([C,f,g], C[f], C[g], C[f @ g])

is_cat = kd.define("is_cat", [C], 
    smt.And(
    smt.ForAll([x,y,z,f,g], C[f], C[g], C[f @ g]),
    smt.ForAll([x], C[f], C[id(dom(f)]
    )
)
is_functor = kd.define("is_functor", [F, C, D],
    is_cat(C),
    is_cat(D),
    smt.ForAll([f], C[f], D[F[f]]),
    smt.ForAll([f, g], C[f], C[g], F[f @ g] == F[f] @ F[g])
    #smt.ForAll([x], C[id(x)], D[F[id(x)]]) ?
)

Functor = kd.Struct("Functor", ("obmap", smt.ArraySort(Ob, Ob)), ("arrmap", smt.ArraySort(Arr, Arr)))
kd.notation.wf.define([F], )

# is it still a functor if just the domain is a functor?
is_functor = kd.define("is_functor", [F, C],
    is_cat(C),
    smt.ForAll([f, g], C[f], C[g], F[f @ g] == F[f] @ F[g])
    smt.ForAll([x], C[id(x)], )
)


smt.ForAll([C,D], 

)



|= ForAll([w, x, y, z, f, g, h],
       Implies(And(hom(x, y)[f], hom(y, z)[g], hom(z, w)[h]),
               comp(comp(f, g), h) == comp(f, comp(g, h))))

In [19]:
from typing import Protocol
from kdrag.all import *
class Category(Protocol):
    Ob : smt.SortRef
    Arr : smt.SortRef
    cod : smt.FuncDeclRef # cod : Arr -> Ob
    dom : smt.FuncDeclRef # dom : Arr -> Ob
    id_ : smt.FuncDeclRef # id : Ob -> Arr
    id_wf : kd.Proof # forall x, dom(id(x)) = x & cod(id(x)) = x
    comp : smt.FuncDeclRef # comp : Arr x Arr -> Arr
    comp_wf : kd.Proof # forall f g, dom(f) = cod(g) => dom(comp(f,g)) == dom(g) & cod(comp(f,g)) == cod(f)
    comp_id : kd.Proof # forall f, comp(f, id(dom(f))) = f 
    id_comp : kd.Proof # forall x, comp(id(cod(f)), f) = f
    comp_assoc : kd.Proof # forall f g h, comp(comp(f,g), h) = comp(f, comp(g,h)) 


#class ACategory(Category):
Ob = smt.DeclareSort("AOb")
Arr = smt.DeclareSort("AArr")
x,y,z = smt.Consts("x y z", Ob)
f,g,h,k = smt.Consts("f g h k", Arr)
cod = smt.Function("cod", Arr, Ob)
dom = smt.Function("dom", Arr, Ob)
hom = kd.define("hom", [x,y], smt.Lambda([f], smt.And(dom(f) == x, cod(f) == y))) # homset
#smt.Const("hom", smt.ArraySort(Ob, Ob, smt.BoolSort()))

# allow assumes to be callable?
f.assumes = hom(x,y)[f] #smt.And(dom(f) == x, cod(f) == y)
g.assumes = hom(y,z)[g] #smt.And(dom(g) == y, cod(g) == z)

all_morph = kd.define("all_morph", [f], smt.Exists([x,y], hom(x,y)[f]))
# We could assume perhaps that categories are derived from ObSets via hom. Distinct categories are disjoint.

# cat = ArraySort(Ob, Ob, Arr, smt.BoolSort())  . But then we can'th ave objects with no arrows. But every object has id.
# So this one set does encode obset and homset
# A functor is a function arr -> arr. The object mapping is derived?

id_ = smt.Function("id", Ob, Arr)

id_wf = kd.axiom(smt.ForAll([x], smt.And(dom(id_(x)) == x, cod(id_(x)) == x)))
comp = smt.Function("comp", Arr, Arr, Arr)
kd.notation.matmul.register(Arr, lambda f,g: comp(f,g))
def CompWF(f,g):
    return dom(f) == cod(g)

comp_wf = kd.axiom(kf.QForAll([f,g], CompWF(f,g), smt.And(dom(comp(f,g)) == dom(g), cod(comp(f,g)) == cod(f)))) # comp_type
comp_id = kd.axiom(smt.ForAll([f], comp(f, id_(dom(f))) == f))
id_comp = kd.axiom(smt.ForAll([f], comp(id_(cod(f)), f) == f))
comp_assoc = kd.axiom(smt.ForAll([f,g,h], comp(comp(f,g), h) == comp(f, comp(g,h))))

def HasType(f, A, B):
    return smt.And(dom(f) == A, cod(f) == B)
has_type = kd.define("has_type", [f, x, y], HasType(f, x, y))
is_iso = kd.define("is_iso", [f], kd.QExists([g], dom(g) == cod(f), cod(g) == dom(f), f @ g == id_(dom(g))), g @ f == id_(cod(g)))

# Dual and Product Category
pair = kd.notation.SortDispatch(name="pair")
pair_ob = smt.Function("pair", Ob, Ob, Ob)
pair.register(Ob, pair_ob)
pair_arr = smt.Function("pair", Arr, Arr, Arr)
pair.register(Arr, pair_arr)

pair_wf = kd.axiom(kd.QForAll([f,g], dom(pair(f,g)) == pair(dom(f), dom(g)), cod(pair(f,g)) == pair(cod(f), cod(g))))
comp_pair = kd.axiom(kd.QForAll([f,g,h,k], dom(f) == cod(h), dom(g) == cod(k), pair(f,g) @ pair(h,k) == pair(f @ h, g @ k)))


ObSet = smt.ArraySort(Ob, smt.BoolSort())
obset = smt.Const("obset", ObSet)
ArrSet = smt.ArraySort(Arr, smt.BoolSort())
arrset = smt.Const("arrset", ArrSet)

Cat = kd.Struct("Cat", ("obset", ObSet), ("arrset", ArrSet), pred=
    lambda cat:smt.And(
    kd.QForAll([x,f], cat.arrset[f], smt.And(cat.obset[dom(f)], cat.obset[cod(f)])),
    kd.QForAll([x], cat.obset[x], cat.arrset[id_(x)]),
    kd.QForAll([f,g], cat.arrset[f], arrset[g], dom(f) == cod(g), cat.arrset[f @ g])
))
cat = smt.Const("cat", Cat)

is_cat = kd.define("is_cat", [cat],
smt.And(
    kd.QForAll([x,f], cat.arrset[f], smt.And(cat.obset[dom(f)], cat.obset[cod(f)])),
    kd.QForAll([x], cat.obset[x], cat.arrset[id_(x)]),
    kd.QForAll([f,g], cat.arrset[f], arrset[g], dom(f) == cod(g), cat.arrset[f @ g])
))


star = smt.Const("star", Ob)
monoid = kd.define("monoid", [], smt.ForAll([x], x == star))

e = kd.define("e", [], id_(star))
monoid_comp_id = kd.prove(kd.QForAll([x], monoid, f @ e == f), by=[comp_id, id_comp, monoid.defn, e.defn])
monoid_id_comp = kd.prove(kd.QForAll([x], monoid, e @ f == f), by=[comp_id, id_comp, monoid.defn, e.defn])
monoid_assoc = kd.prove(kd.QForAll([f,g,h], monoid, (f @ g) @ h == f @ (g @ h)), by=[comp_assoc, monoid.defn])







NameError: name 'kf' is not defined

And(ForAll([x, y, z],
           mul(mul(x, y), z) == mul(x, mul(y, z))),
    ForAll(x, mul(x, e) == x),
    ForAll(x, mul(e, x) == x))

# Linear Space

# set zf
Sets as my generics. Is there going to be a inconsinstnecy problem if I start asserting that there exists sets that represent every value?
Typeable, dynamic, etc. A serialization format.


Nat would be the first place to do it.
```
kd.define()
kd.define("IntZF", [], pair( , omega)) # set that represents
kd.define("to_int, [s], smt.If(elem(s, IntZF),    , undef))
kd.define("to_zf), [n], smt.If(n == 0, emp, succ(n-1),)  


# This could be really interesting. Calrifying of family / universe issues
dt = ZFInductive(name)
ZFInductive()

def disjoint_sum(sets):
    return union( (sing(zf_int(i)), set) for n,s in enumerate(sets))


```


Why not do a dependent type theory? Sear? Topos
Elem = DeclareSort()





Maybe if I'd used elts more with built in set operations some stuff could be more automatic?

https://www.isa-afp.org/sessions/hol/#Fun.html pull other stuff from isabelle. Work backwards?

rework triggers

map_pos on rewrite.
rw could work under binders
rw in prove

https://isabelle.in.tum.de/dist/Isabelle2025/doc/logics-ZF.pdf

https://en.wikipedia.org/wiki/Tarski%E2%80%93Grothendieck_set_theory

In [ ]:
univ = smt.Function("univ", ZFSet, ZFSet)
elem_univ = kd.axiom(kd.QForAll([x], elem(x, univ(x))))
univ_subset = kd.axiom(kd.QForAll([x, y], y <= x, elem(y, univ(x))))
univ_pow = kd.axiom(kd.QForAll([x], elem(pow(x), univ(x))))
univ_card = kd.axiom(kd.QForAll([x,y], card(x) <= card(univ(y)), elem(x, univ(y))))

card = smt.Function("card", ZFSet, ZFSet) # canonical element of each bijection class

In [33]:
from kdrag.all import *
from kdrag.theories.logic.zf import *


@kd.PTheorem(smt.ForAll([x,a], (x <= sing(a)) == smt.Or(x == sing(a), x == emp)))
def le_sing(l):
    l.unfold(le)
    x,a = l.fixes()
    l.cases(x == emp)
    
    l.auto(by=[elem_sing, elem_emp])

    l.rw(-1).simp()
    l.split().intros()
    l.rw(ext_ax)
    x1 = l.fix()
    l.instan(-1, x1)
    l.rw(elem_sing, at=-1)
    l.rw(elem_sing)
    l.split()
    l.intros()
    l.auto()
    l.intros()
    l.rw(-1)
    #l.auto(by=[elem_sing, elem_emp])







Next Goal:
 [x!1743, a!1744, x!1748];
[(x!1743 == emp) == False,
 ForAll(x, Or(Not(elem(x, x!1743)), elem(x, sing(a!1744)))),
 Or(Not(elem(x!1748, x!1743)), x!1748 == a!1744),
 x!1748 == a!1744]
?|= elem(a!1744, x!1743)


In [ ]:
    #l.auto(by=[elem_sing, elem_emp])
    l.split()
    l.intros()
    #l.have(elem(a, sing(a)), by=[elem_sing])
    #l.have(smt.Not(elem(x, emp)), by=[elem_emp])
    #l.instan(0, a)
    l.cases(x == emp)
    l.case().auto(by=[elem_emp])
    l.rw(-1).simp()
    x1 = l.rw(ext_ax).fix()
    l.instan(0, x1)
    l.split()
    l.auto()
    l.rw(elem_sing)
    l.have(sing(a) != emp, by=[sing_not_emp])
    #l.auto()
    #l.auto(by=[elem_sing, elem_emp])

#kd.search(emp.decl())


In [7]:
@kd.PTheorem(smt.ForAll([a], pow(sing(a)) == sing(a) | emp))
def pow_sing(l):
    a = l.fix()
    l.rw(ext_ax).fix()
    l.rw(elem_pow).rw(elem_union).rw(elem_emp)

Next Goal:
 [a!399, x!402];
[] ?|= le(x!402, sing(a!399)) ==
Or(elem(x!402, sing(a!399)), False)


In [ ]:

    #l.auto(by=[inter_le])

#kd.search(le)

In [7]:
from kdrag.all import *
from kdrag.theories.logic.zf import *


pow(upair(a,b)) == FinSet(emp, sing(a), sing(b), upair(a,b))
pow(sing(a)) == sing(a) | emp
pow(a & b) == pow(a) & pow(b)
pow(a | b) <= pow(a) | pow(b)


mul = kd.notation.mul.define([A,B], sep(pow(pow(A | B)), smt.Lambda([p], kd.QExists(elem(a,A), elem(b,B), p == pair(a,b)))))

elem_mul = kd.prove(smt.ForAll([A,B,a,b], elem(pair(a,b), A * B)) == smt.And(elem(a,A), elem(b,B)))
elem_mul = kd.prove(smt.ForAll([A,B,p], elem(p, A * B)) == smt.And(elem(fst(p), A), elem(snd(b),B))) # this one is more general.
elem_mul = kd.prove(smt.ForAll([A,B,p], elem(p, A * B)) == smt.Exists([a,b], elem(a,A), elem(b,B), p == pair(a,b)))
elem_mul_is_pair = kd.prove(kd.QForAll([A,B,p], elem(p, A * B), is_pair(p)))


(A | B) * X == (A * X) | (B * X)
X * (A | B) == (X * A) | (X * B)
(A & B) * X == (A * X) & (B * X)
X * (A & B) == (X * A) & (X * B)
(A - B) * X == (A * X) - (B * X)

A * emp == emp
emp * A == emp

smt.Implies(smt.And(A <= X, B <= Y), A * B <= X * Y)

dom = kd.define("dom", [R], sep(bigunion(bingunion(R)), smt.Lambda([a], kd.QExists([b], elem(pair(a,b)), R))))
ran = kd.define("ran", [R], sep(bigunion(bingunion(R)), smt.Lambda([b], kd.QExists([a], elem(pair(a,b)), R))))
#dom = kd.define("dom", [R], )
idrel = kd.define("idrel", [A], sep(A * A, smt.Lambda([a,b], a == b)))

has_type = kd.define("has_type", [R, A, B],  R <= A * B)
#is_refl = kd.define("is_refl", [R, X], has_type(R,X,X) & kd.QForAll([x], elem(x,X), elem(pair(x,x), R)))
is_refl = kd.define("is_refl", [R], kd.QForAll([x], smt.Exists([y], elem(pair(x,y), R) | elem(pair(y,x), R)), elem(pair(x,x), R)))
is_symm = kd.define("is_symm", [R], smt.ForAll([a,b], elem(pair(a,b), R), elem(pair(b,a), R)))
is_trans = kd.define("is_trans", [R], kd.QForAll([a,b,c], elem(pair(a,b), R) & elem(pair(b,c), R), elem(pair(a,c), R)))
is_equiv = kd.define("is_equiv", [R], smt.And(is_refl(R), is_symm(R), is_trans(R)))

is_equiv(idrel(A))
is_equiv(X * X)


is_partition = kd.define("is_partition", [P, A], smt.And(biginter(P) == emp, bigunion(P) == A))


is_fun = kd.define("is_fun", [R], kd.QForAll([a,b,c], elem(pair(a,b), R), elem(pair(a,c), R), b == c))
is_total = kd.define("is_total", [R, A], kd.QForAll([a], elem(a,A), smt.Exists([b], elem(pair(a,b), R))))

is_fun(idrel(A))

Family = smt.ArraySort(ZFSet, ZFSet)
# is internal
#is_intfun = kd.define("is_intfun", [F], smt.Exists([R], is_fun(R), smt.ForAll([x], F[x] == )

#is_set = kd.define("is_set", [P], smt.Exists([A], smt.ForAll([x], elem(x, A) == P[x])))





le(pow(union(a!0, b!1)), union(pow(a!0), pow(b!1)))

In [ ]:

@kd.Theorem(smt.Not(gencomp))
def russell(l):
    l.unfold(is_set)
    l.unfold(reflects)
    l.intros()
    l.instan(0, smt.Lambda([x], smt.Not(elem(x, x))))
    A = l.obtain(-1)
    l.instan(-1, A)
    l.auto()


In [ ]:
sing(a) | sing(b) == upair(a,b)
sing(a) | upair(a,b) == upair(a,b)

In [ ]:



    #l.unfold(pair)
    #l.rw(bigunion_upair)
    #l.have(upair(sing(a), upair(a,b)) != emp, by=[upair_not_emp])
    #l.rw(sing_union_upair)
    #l.have(smt.Or(pick(upair(a,b)) == a, pick(upair(a,b)) == b), by=[pick_upair])
    #l.cases(pick(upair(a,b)) == a)
    #l.have()
#kd.search(bigunion)


In [33]:
kd.search(union)

{('__main__.union_comm',
  |= ForAll([a!0, b!1], union(a!0, b!1) == union(b!1, a!0))): (),
 ('__main__.elem_union',
  |= ForAll([x, A, B],
         elem(x, union(A, B)) == Or(elem(x, A), elem(x, B)))): (),
 ('__main__.union_assoc',
  |= ForAll([a!0, b!1, c!2],
         union(union(a!0, b!1), c!2) ==
         union(a!0, union(b!1, c!2)))): (),
 ('__main__.union_idem', |= ForAll(a!0, union(a!0, a!0) == a!0)): (),
 ('__main__.le_union',
  |= ForAll([a!0, b!1, c!2], le(a!0, union(a!0, b!1)))): (),
 ('__main__.inter_union_distr',
  |= ForAll([a!0, b!1, c!2],
         inter(a!0, union(b!1, c!2)) ==
         union(inter(a!0, b!1), inter(a!0, c!2)))): (),
 ('__main__.union_inter_distr',
  |= ForAll([a!0, b!1, c!2],
         inter(union(a!0, b!1), c!2) ==
         union(inter(a!0, c!2), inter(b!1, c!2)))): (),
 ('__main__.bigunion_upair',
  |= ForAll([a!0, b!1],
         bigunion(upair(a!0, b!1)) == union(a!0, b!1))): (),
 ('kdrag.theories.logic.zf.union_comm',
  |= ForAll([a!0, b!1], union(a!0

In [15]:

@kd.tactics.PTheorem(kd.QForAll([a], a != emp, kd.QExists([c], c != emp, elem(c, a)), a != sing(emp), bigunion(a) != emp))
def bigunion_not_emp(l):
    a = l.fix()
    l.intros()
    l.split(at=0)
    c = l.obtain(1)
    l.intros()
    l.rw(ext_ax, at=-1)
    l.have(elem(pick(a), a), by=[elem_pick])
    #l.have(elem(kd.QExists([d], elem(d, a)))
    #l.have(elem(pick(a), bigunion(a)), by=[bigunion_ax, elem_pick])
    

    
    #l.have(elem(pick(a), bigunion(a)))


TimeoutError: Timeout. Maybe you have given `prove` too many or not enough lemmas?

In [21]:
smt.is_distinct(bigunion(a) != emp)

True





#kd.search(bigunion)

In [16]:
kd.search(bigunion)



{('__main__.bigunion_ax',
  |= ForAll([A, x],
         elem(x, bigunion(A)) ==
         (Exists(B, And(elem(B, A), elem(x, B)))))): (),
 ('__main__.elem_bigunion_upair',
  |= ForAll([x, A, B],
         elem(x, bigunion(upair(A, B))) ==
         Or(elem(x, A), elem(x, B)))): (),
 ('kdrag.theories.logic.zf.bigunion_ax',
  |= ForAll([A, x],
         elem(x, bigunion(A)) ==
         (Exists(B, And(elem(B, A), elem(x, B)))))): (),
 ('kdrag.theories.logic.zf.elem_bigunion_upair',
  |= ForAll([x, A, B],
         elem(x, bigunion(upair(A, B))) ==
         Or(elem(x, A), elem(x, B)))): ()}

In [ ]:


    #l.auto(by=[pick_upair])
    #l.unfold   #l.rw(elem_sing)
    #l.auto()


In [ ]:

l = kd.Lemma(smt.ForAll([a,b,c,d], (pair(a,b) == pair(c,d)) == smt.And(a == c, b == d)))



kd.Lemma(fst(pair(a,b)) == a)

kd.Lemma(snd(pair(a,b)) == b)

@Theorem()

In [45]:
ProofState = kd.tactics.ProofState

_TRUE = kd.kernel.prove(smt.BoolVal(True))


def Theorem(
    goal: smt.BoolRef,
) -> Callable[[Callable[[ProofState], None]], kd.kernel.Proof]:
    def res(f: Callable[[ProofState], None]) -> kd.Proof:
        l = kd.Lemma(goal)
        f(l)
        pf = l.qed()
        object.__setattr__(pf, "__doc__", getattr(f, "__doc__", None))
        object.__setattr__(pf, "__module__", getattr(f, "__module__", None))
        object.__setattr__(pf, "__name__", getattr(f, "__name__", None))
        object.__setattr__(pf, "__qualname__", getattr(f, "__qualname__", None))
    return res


x, y, z = smt.Ints("x y z")

@Theorem(x + 1 > x)
def inc_ge(l):
    "increment the stuff"
    l.auto()

def check(p: kd.Proof) -> kd.Proof:
    return p

assert inc_ge is not None
help(check(inc_ge))

Help on Proof in module __main__:

inc_ge = |= x + 1 > x
    incremenet the stuff



In [ ]:
@Lemma()
def elem_union(l):
    a,b,c,d = l.fix()
    l.auto()
    l.unfold(elem)
    l.rw(le)
    l.auto()
    


In [31]:
smt.ForAll([a,x], elem(x,a), patterns=[elem(x,a)])

ForAll([a!0, x], elem(x, a!0))

In [2]:
from kdrag.all import *
from kdrag.theories.logic.zf import *

#A, B, x, y, z = smt.Consts("A B x y z", zf.ZFSet)

#union_comm = #kd.prove(a | b == b | a, by=[bigunion_ax, upair_ax, ext_ax(a | b, b | a)])

l = kd.Lemma(a | b == b | a)
l.unfold(union)
l.rw(ext_ax)
_x = l.fix()
l.auto(by=[upair_ax, bigunion_ax])
union_comm = l.qed().forall([a,b])

l = kd.Lemma(smt.ForAll([a,b,c], (a | b) | c == a | (b | c)))
_a, _b,  _c = l.fixes()
#l.unfold(union)
l.rw(ext_ax)
_x = l.fix()
l.rw(elem_union)
l.rw(elem_union)
l.rw(elem_union)
l.rw(elem_union)
l.auto()
union_assoc = l.qed()
#l.rw(bigunion_ax)
#l.rw(bigunion_ax)
#l
#l.split()
#l.intros()
#_B = l.obtain(0)
#l.rw(upair_ax, at=0)
#l.rw(upair_ax)
#l.auto(by=[upair_ax, upair_ax(a,b, _x), upair_ax(b,c, _x), bigunion_ax])
#union_assoc = l.qed().forall([a,b,c])


#l
#l.auto(by=[sep_ax, upair_ax, bigunion_ax])
#inter_distr = l.qed().forall([a,b,c])




#bigunion = smt.Function("bigunion", zf.Set, zf.Set)
#bigunion_ax = kd.axiom()


In [ ]:
kd.Lemma(smt.Implies(A == emp, pick(A)))

In [ ]:
from kdrag.all import *
from kdrag.theories.logic.zf import *

#l.newgoal(elem)

#.intros()
#l
#l.have(smt.ForAll([_x], ))
#l.rw(ext_ax, at=-1)
#l.revert(0)

#l.rw(emp_exists_elem)
#l.simp()
#l.exists(pick(_A))
#l.apply(elem_pick)

#l.auto(by=[elem_pick(_x, _A)])
#l.rw(elem_pick)

In [72]:
l = kd.Lemma(smt.ForAll([a,b,c], a & (b | c) == (a & b) | (a & c)))
l.fixes()
l.rw(ext_ax)
l.fix()
l.rw(elem_inter)
l.rw(elem_union)
l.rw(elem_inter)
l.rw(elem_union)
l.rw(elem_inter)
l.auto()
inter_union_distr = l.qed()


In [ ]:
from kdrag.all import *
from kdrag.theories.logic.zf import *

l = kd.Lemma(smt.ForAll([A], (A == emp) == smt.Not(smt.Exists([x], elem(x, A)))))
_A = l.fix()
l.split()
with l.sub() as l1:
    l1.intros()
    l1.intros()
    _x = l1.obtain(-1)
    l1.show(smt.BoolVal(False), by=[ext_ax, elem_emp])

with l.sub() as l2:
    l2.intros()
    l2.rw(ext_ax)
    _x = l2.fix()
    l2.apply(0)
    l2.exists(_x)
    l2.have(elem(_x, emp) == False, by=[elem_emp(_x)])
    l2.have(elem(_x, _A), by=[])

    l2.auto()
emp_exists_elem = l.qed()
#l.have(smt.ForAll([x], elem(x,A)), by=[ext_ax(A, emp)])


In [4]:
l = kd.Lemma(smt.ForAll([A], smt.Implies((A != emp), smt.Exists([x], elem(x, A)))))
_A = l.fix()
l.auto(by=[emp_exists_elem(_A)])
l.qed()
not_emp_exists_elem = l.qed()

|= ForAll(A, Implies(A != emp, Exists(x, elem(x, A))))

In [8]:
l = kd.Lemma(smt.ForAll([a,b,c], (a | b) & c == (a & c) | (b & c)))
l.fixes()
l.auto(by=[inter_comm, inter_union_distr])
union_inter_distr = l.qed()
#l.rw(inter_union_distr)
#l.rw(inter_comm)

In [14]:
l = kd.Lemma(smt.ForAll([a,b], a & b <= a))
l.fixes()
l.unfold(le)
l.fix()
l.rw(elem_inter)
l.auto()
inter_le = l.qed()

In [16]:
l = kd.Lemma(smt.ForAll([a,b,c], a <= a | b))
l.fixes()
l.unfold(le)
l.fix()
l.rw(elem_union)
l.auto()
le_union = l.qed()

In [62]:
l = kd.Lemma(smt.ForAll([x,A,B], elem(x, A & B) == (elem(x,A) & elem(x,B))))
l.fixes()
l.unfold(inter)
l.rw(sep_ax)
l.auto()
elem_inter = l.qed()

In [66]:
l = kd.Lemma(smt.ForAll([A,B], A & B == B & A))
l.fixes()
l.rw(ext_ax)
l.fix()
l.rw(elem_inter)
l.rw(elem_inter)
l.auto()
l.qed()

|= ForAll([A, B], inter(A, B) == inter(B, A))

In [67]:
l = kd.Lemma(smt.ForAll([a,b,c], (a & b) & c == a & (b & c)))
l.fixes()
l.rw(ext_ax)
l.fix()
l.rw(elem_inter)
l.rw(elem_inter)
l.rw(elem_inter)
l.rw(elem_inter)
l.auto()
inter_assoc = l.qed()

|= ForAll([a!0, b!1, c!2],
       inter(inter(a!0, b!1), c!2) ==
       inter(a!0, inter(b!1, c!2)))

In [ ]:

    #_B2 = c2.obtain(0)
    #c2.rw(upair_ax, at=0)
    #c2.auto()

#l.rw(ext_ax)


In [ ]:
elem_union = kd.prove(
    elem(c, union(a, b)) == (elem(c, a) | elem(c, b))),
    by=[elem_bigunion_upair

In [ ]:
l = kd.Lemma((a & b) & c == a & (b & c))
l.unfold(inter)
l.rw(ext_ax)
l.auto(by=[sep_ax])
inter_assoc = l.qed().forall([a,b,c])

l = kd.Lemma(a & b == b & a)
l.unfold(inter)
l.rw(ext_ax)
l.auto(by=[sep_ax])
inter_comm = l.qed().forall([a,b])

In [100]:
l = kd.Lemma(elem(x, a | b) == (elem(x,a) | elem(x,b)))
l.unfold(union)
l.rw(bigunion_ax)
l.split()
l.intros()
_B = l.obtain(0)
l.rw(upair_ax, at=0)
l.auto()

l.intros()
l.cases(elem(x, a))
l.exists(a)
l.rw(upair_ax)
l.auto()
l.exists(b)
l.rw(upair_ax)
l.auto()
l.qed()


|= elem(x, union(a!0, b!1)) == Or(elem(x, a!0), elem(x, b!1))

In [104]:
l = kd.Lemma(elem(x, a & b) == (elem(x,a) & elem(x,b)))
l.unfold(inter)
l.rw(sep_ax)
l.auto()
l.qed()

|= elem(x, inter(a!0, b!1)) == And(elem(x, a!0), elem(x, b!1))

In [72]:


l = kd.Lemma(a & (b | c) == (a & b) | (a & c))
l.unfold(inter, union)
l.rw(ext_ax)
_x = l.fix()
l.rw(bigunion_ax)
l.rw(sep_ax)
l.simp()
l.rw(bigunion_ax)
l.split()
l.intros()
l.split(at=-1)

_B = l.obtain(0)
l.split(0)
#l.exists(_B)


[x!3390, f!3398(b!1, c!2)];
[elem(f!3398(b!1, c!2), upair(b!1, c!2)),
 elem(x!3390, f!3398(b!1, c!2)),
 elem(x!3390, a!0)]
?|= Exists(B,
       And(elem(B,
                upair(sep(a!0, Lambda(x, elem(x, b!1))),
                      sep(a!0, Lambda(x, elem(x, c!2))))),
           elem(x!3390, B)))

In [106]:
l = kd.Lemma(a & (b | c) == (a & b) | (a & c))
l.rw(ext_ax)
_x = l.fix()
l.rw(union_ax)
l.rw(inter_ax)
l.auto()

[x!3555];
[]
?|= elem(x!3555, inter(a!0, union(b!1, c!2))) ==
elem(x!3555, union(inter(a!0, b!1), inter(a!0, c!2)))

In [ ]:
l.rw(upair_ax)
l.split()
l.rw(upair_ax, at=0)
l.left()
l.rw(ext_ax)
_x1 = l.fix()
l.rw(sep_ax)
l.apply(-1)
l.rw(ext_ax)
_x2 = l.fix()
l.rw(sep_ax)
l.simp()
#l.auto()
#l.auto(by=[sep_ax])

Uh oh. Is it ok that z3 doesn't know that it needs to skolem though freshvars?

Implicit z3 needs to solve forall(xs, exists(zs, P)). Not(forall, (exists, )) == exists xs, forall za Not(P) is unsat  
But it thinks it's proving exists(xs, exists(zs, P)). Not(exists(xs, exists(zs, P))) ==  forall xs, forall zs, Not(P) is unsat., A stronger statement or is it weaker?




# SEAR
https://ncatlab.org/nlab/show/SEAR

In [ ]:
Elt = smt.DeclareSort("SEARElt")
Set = smt.DeclareSort("SEARSet")
Rel = smt.DeclareSort("SEARRel")

elts = smt.Function("elts", Set, smt.ArraySort(Elt, smt.BoolSort()))
rels = smt.Function("rels", Set, Set, smt.ArraySort(Rel, smt.BoolSort()))
holds = smt.Function("holds", Rel, smt.ArraySort(Elt, Elt, smt.BoolSort()))

kd.notation.getitem.register(Set, lambda S, x: elts(S)[x])
# actually overload based on number of args. 
kd.notation.getitem.register(Rel, lambda R, args: holds(R)[args] if len(args) == 2 else holds(R)[tt, args])

xA,yA,zA = smt.Consts("xA yA zA xB yB", Elt)
A,B,C = smt.Consts("A B C", Set)
xA.assumes = elts(A)[xA]
yA.assumes = elts(A)[yA]
zA.assumes = elts(A)[zA]
xB.assumes = elts(B)[xB]
yB.assumes = elts(B)[yB]

phi = smt.Const("phi", Rel)
phi.assumes = rels(A,B)[phi]


P,Q,R = smt.Consts("P Q R", smt.ArraySort(Elt, Elt, smt.BoolSort()))


nontriv = smt.Const("nontriv", Set)
elts_nontriv = smt.Exists([x], elts(nontriv)[x])

compr = smt.Function("compr", smt.ArraySort(Elt, Elt, smt.BoolSort()), Rel)
#holds_compr = kd.axiom(kd.QForAll([P,A,B,x,y], elts(A)[x], elts(B)[x], holds(compr(P))[x,y] == P[x,y]))
holds_compr = kd.axiom(kd.QForAll([P,A,B,xA,yB], holds(compr(P))[x,y] == P[x,y]))


is_fun = kd.define("is_fun", [phi], kd.QForAll([A,B], rels(A,B)[phi], smt.ForAll([x], elts(A)[x], smt.ExistsUnique([y], elts(B)[y], holds(phi)[x,y]))))

tt = smt.Const("tt", Elt)





In [ ]:
from kdrag.all import *

# https://ncatlab.org/nlab/show/ETCS
Set = smt.DeclareSort("ETCSSet")
Fun = smt.DeclareSort("ETCSFun")
hom = smt.Function("hom", Set, Set, smt.ArraySort(Fun, smt.BoolSort()))
A,B,C = smt.Consts("A B C", Set)
f,g,h,k = smt.Consts("f g h k", Fun)


id = smt.Function("id", Set, Fun)
hom_id = kd.axiom(smt.ForAll([A], hom(A,A)[id(A)]))


